# Video-RAG: Semantic Search and Summarization Demo


This project presents a **modular, end-to-end pipeline** for semantic video retrieval and question answering over educational video content. Built entirely for **local execution**, the system allows users to query a set of lecture videos in natural language and receive:

- 🔍 **Relevant transcript segments** ranked by semantic similarity (via Qdrant)
- 🎬 **Contextually-aligned video clips** extracted automatically using timestamp metadata
- 🧠 **Concise, structured summaries** generated by a local LLM (Gemma 3 via Ollama)

The architecture integrates several key components:
- **Sentence-transformer-based embedding models** (`all-MiniLM-L6-v2`) for semantic representation
- **MongoDB** for structured storage of subtitles, questions, and video metadata
- **Qdrant** as the high-performance vector database for nearest-neighbor search
- **Gradio** for an intuitive interface to run retrieval, playback, and summarization interactively
- **FFmpeg** for lightweight, local clip slicing without re-encoding

The system is optimized for **retrieval-augmented generation (RAG)** use cases over multimedia corpora. All components run offline, enabling secure deployments in privacy-sensitive or compute-constrained environments.

---


## 📦 Requirements

To run this project locally, you’ll need the following Python packages:

- **Natural Language Processing**: `sentence-transformers`, `transformers`, and `tokenizers` for embedding and summarization.
- **Topic Modeling**: `BERTopic`, `hdbscan`, and `umap-learn` for clustering and label discovery.
- **Vector Search**: `qdrant-client` connects to the local Qdrant instance for fast semantic retrieval.
- **Interface**: `gradio` provides a clean front-end for query, summary, and clip playback.
- **Database**: `pymongo` is used to read/write structured sentence and video metadata.
- **Video**: `ffmpeg-python` allows optional FFmpeg binding in Python for video slicing.

> 💡 Make sure [FFmpeg](https://ffmpeg.org/download.html) and [Ollama](https://ollama.com) are installed separately on your system for video slicing and local LLM usage.


In [ ]:
!pip install transformers
!pip install open-clip-torch
!pip install datasets webdataset pymongo opencv-python decord
!pip install pyAV
!pip install --upgrade av
!pip install webdataset datasets
!pip install webvtt
!pip install ollama
!pip install sentence-transformers
!pip install bertopic
!pip install numpy==1.26.4
!pip install hf_xet
!pip install qdrant-client
!pip install gradio


To begin, we authenticate with Hugging Face and load the aegean-ai/ai-lectures-spring-24 dataset. This dataset contains AI lecture videos along with rich metadata including transcripts and subtitles. We use the Hugging Face Datasets library to fetch and preview the video data (stored as MP4s), which we later process for slicing, summarization, and semantic retrieval tasks.

In [ ]:
from huggingface_hub import login

login('MASKED')

In [ ]:
from datasets import load_dataset

# Specify the directory where you want to store the dataset
data_dir = r"C:\Users\sahil\Downloads\AIVideos"

# Load the dataset (use the correct split name as per the structure)
dataset = load_dataset("aegean-ai/ai-lectures-spring-24")

In [45]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['mp4', 'info.json', 'en.vtt', 'json', '__key__', '__url__'],
        num_rows: 8
    })
})


In [46]:
example = dataset['train'][0]
print("Keys:", example.keys())
print("MP4 type:", type(example['mp4']))
print("MP4 content sample:", str(example['mp4'])[:100])

Keys: dict_keys(['mp4', 'info.json', 'en.vtt', 'json', '__key__', '__url__'])
MP4 type: <class 'bytes'>
MP4 content sample: b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x047/moov\x00\x00\x00lmvhd\x00\x00\x00\x


### 🔧 Converting Hugging Face Dataset to WebDataset Format

To streamline large-scale video processing and enable efficient streaming for downstream ML tasks, we convert the `ai-lectures-spring-24` dataset from Hugging Face into the WebDataset format. Each shard in this format is a `.tar` archive that encapsulates multiple lecture samples—each containing:

- 🎞️ **`.mp4`**: The lecture video segment  
- 💬 **`.vtt`**: The subtitle track  
- 📄 **`.json` / `info.json`**: Rich metadata for alignment and context

This structure is ideal for scalable data ingestion in GPU pipelines, ensures fast random access, and maintains semantic grouping of multimedia resources. By sharding the data (e.g., 2 samples per shard), we enable modular processing and facilitate reproducible experimentation in high-throughput environments.


In [47]:
from datasets import load_dataset
import json
import os
import tarfile
import io

# Configuration
output_dir = r"C:\Users\sahil\Downloads\AIVideos\webdataset"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
dataset = load_dataset("aegean-ai/ai-lectures-spring-24")

def create_shards():
    samples_per_shard = 2
    total_samples = len(dataset['train'])
    
    for shard_idx in range(0, (total_samples + samples_per_shard - 1) // samples_per_shard):
        shard_filename = os.path.join(output_dir, f"shard-{shard_idx:06d}.tar")
        print(f"Creating {shard_filename}")
        
        with tarfile.open(shard_filename, "w") as tar:
            start_idx = shard_idx * samples_per_shard
            end_idx = min(start_idx + samples_per_shard, total_samples)
            
            for idx in range(start_idx, end_idx):
                example = dataset['train'][idx]
                
                # Use idx as key if __key__ doesn't exist
                key = example.get('__key__', f"sample-{idx:06d}")
                
                # Add JSON data
                if 'json' in example and example['json']:
                    json_data = json.dumps(example['json']).encode('utf-8')
                    json_info = tarfile.TarInfo(f"{key}.json")
                    json_info.size = len(json_data)
                    tar.addfile(json_info, io.BytesIO(json_data))
                
                # Add mp4 data if available
                if 'mp4' in example and example['mp4']:
                    mp4_data = example['mp4']
                    mp4_info = tarfile.TarInfo(f"{key}.mp4")
                    mp4_info.size = len(mp4_data)
                    tar.addfile(mp4_info, io.BytesIO(mp4_data))
                
                # Add VTT subtitle if available
                if 'en.vtt' in example and example['en.vtt']:
                    vtt_data = example['en.vtt']
                    # Convert to bytes if it's a string
                    if isinstance(vtt_data, str):
                        vtt_data = vtt_data.encode('utf-8')
                    vtt_info = tarfile.TarInfo(f"{key}.vtt")
                    vtt_info.size = len(vtt_data)
                    tar.addfile(vtt_info, io.BytesIO(vtt_data))
                
                # Add info.json if available
                if 'info.json' in example and example['info.json']:
                    info_data = example['info.json']
                    # Convert dictionary to JSON string and encode
                    if isinstance(info_data, dict):
                        info_data = json.dumps(info_data).encode('utf-8')
                    # Convert string to bytes
                    elif isinstance(info_data, str):
                        info_data = info_data.encode('utf-8')
                    info_file = tarfile.TarInfo(f"{key}.info.json")
                    info_file.size = len(info_data)
                    tar.addfile(info_file, io.BytesIO(info_data))
                
                print(f"Processed {key}")

if __name__ == "__main__":
    create_shards()
    print(f"Shards created in {output_dir}")

Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000000.tar
Processed videos/9CGGh6ivg68/9CGGh6ivg68
Processed videos/WXoOohWU28Y/WXoOohWU28Y
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000001.tar
Processed videos/TV-DjM8242s/TV-DjM8242s
Processed videos/rCVlIVKqqGE/rCVlIVKqqGE
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000002.tar
Processed videos/lb_5AdUpfuA/lb_5AdUpfuA
Processed videos/FCQ-rih6cHY/FCQ-rih6cHY
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000003.tar
Processed videos/eQ6UE968Xe4/eQ6UE968Xe4
Processed videos/eFgkZKhNUdM/eFgkZKhNUdM
Shards created in C:\Users\sahil\Downloads\AIVideos\webdataset


### 🎥 Inspecting and Extracting Frames from Videos

We leverage PyTorch’s `torchvision.io.VideoReader` to directly stream and inspect video content from the dataset. The pipeline includes:

- Extracting the **first 10 frames** as raw tensors, which is useful for quick previews or sanity checks.
- Looping through the dataset to obtain **video metadata**, especially the duration, to support alignment with subtitles or timestamp-based slicing.

This minimal example illustrates how to interact with Hugging Face-stored videos at a low level for tasks like frame sampling, segment annotation, and duration-based slicing.


In [ ]:
import torch
import torchvision.io as io
import numpy as np
import itertools

video_reader = dataset[0]["mp4"]  # Get the VideoReader object

# Extract frames as tensors
frames = [frame["data"] for frame in itertools.islice(video_reader, 10)]  
frames = torch.stack(frames)  # Convert to tensor (num_frames, channels, height, width)

print(frames.shape)  

torch.Size([10, 3, 1080, 1920])


In [ ]:
from torchvision.io import VideoReader

# Loop through the dataset
for i, example in enumerate(dataset):
    # Get the video object
    video = example["mp4"]
    
    # Use the get_metadata() method to get the metadata
    metadata = video.get_metadata()
    
    # Extract video duration
    duration = metadata["video"]["duration"][0]  # Duration in seconds
    print(f"Video {i} duration: {duration} seconds")



Video 0 duration: 284.7 seconds
Video 1 duration: 908.7666666666667 seconds
Video 2 duration: 2376.2 seconds
Video 3 duration: 841.9333333333333 seconds
Video 4 duration: 501.1333333333333 seconds
Video 5 duration: 1532.9333333333334 seconds
Video 6 duration: 3101.6666666666665 seconds
Video 7 duration: 1602.7666666666667 seconds


### 🧱 WebDataset Conversion with Key Sanitization

To ensure efficient downstream streaming and processing, we convert the Hugging Face AI lecture dataset into a **WebDataset** format. The process includes:

- **Sanitizing keys** to flatten directory-style identifiers (e.g., `videos/abc:def` → `videos_abc_def`) for cross-platform compatibility.
- Packaging each sample (video, subtitles, metadata, and annotations) into `.tar` shards, with a configurable number of samples per shard.
- Automatically handling different formats for video and text fields (`.mp4`, `.vtt`, `.json`, and `.info.json`), including fallback checks for raw bytes vs. paths.

This conversion is a crucial step for scalable training and inference pipelines, enabling fast random access and sharded streaming across distributed systems.


In [ ]:
from datasets import load_dataset
import json
import os
import tarfile
import io

# Configuration
output_dir = r"C:\Users\sahil\Downloads\AIVideos\webdataset"
os.makedirs(output_dir, exist_ok=True)

# Load dataset
dataset = load_dataset("aegean-ai/ai-lectures-spring-24")

def sanitize_key(key):
    """Convert directory-style keys to flat filenames"""
    return key.replace("/", "_").replace(":", "_")

def create_shards():
    samples_per_shard = 2
    total_samples = len(dataset['train'])
    
    for shard_idx in range(0, (total_samples + samples_per_shard - 1) // samples_per_shard):
        shard_filename = os.path.join(output_dir, f"shard-{shard_idx:06d}.tar")
        print(f"Creating {shard_filename}")
        
        with tarfile.open(shard_filename, "w") as tar:
            start_idx = shard_idx * samples_per_shard
            end_idx = min(start_idx + samples_per_shard, total_samples)
            
            for idx in range(start_idx, end_idx):
                example = dataset['train'][idx]
                original_key = example.get('__key__', f"sample-{idx:06d}")
                key = sanitize_key(original_key)
                
                # Add JSON data
                if 'json' in example:
                    json_data = json.dumps(example['json']).encode('utf-8')
                    info = tarfile.TarInfo(f"{key}.json")
                    info.size = len(json_data)
                    tar.addfile(info, io.BytesIO(json_data))
                
                # Add MP4 data
                if 'mp4' in example and example['mp4']:
                    mp4_data = example['mp4']
                    if isinstance(mp4_data, str):  # If paths are stored instead of bytes
                        with open(mp4_data, "rb") as f:
                            mp4_data = f.read()
                    info = tarfile.TarInfo(f"{key}.mp4")
                    info.size = len(mp4_data)
                    tar.addfile(info, io.BytesIO(mp4_data))
                
                # Add VTT subtitles
                if 'en.vtt' in example:
                    vtt_data = example['en.vtt']
                    if isinstance(vtt_data, str):
                        vtt_data = vtt_data.encode('utf-8')
                    info = tarfile.TarInfo(f"{key}.vtt")
                    info.size = len(vtt_data)
                    tar.addfile(info, io.BytesIO(vtt_data))
                
                # Add info.json
                if 'info.json' in example:
                    info_data = example['info.json']
                    if isinstance(info_data, dict):
                        info_data = json.dumps(info_data)
                    if isinstance(info_data, str):
                        info_data = info_data.encode('utf-8')
                    info = tarfile.TarInfo(f"{key}.info.json")
                    info.size = len(info_data)
                    tar.addfile(info, io.BytesIO(info_data))
                
                print(f"Processed {original_key} -> {key}")

if __name__ == "__main__":
    create_shards()
    print(f"Shards created in {output_dir}")

Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000000.tar
Processed videos/9CGGh6ivg68/9CGGh6ivg68 -> videos_9CGGh6ivg68_9CGGh6ivg68
Processed videos/WXoOohWU28Y/WXoOohWU28Y -> videos_WXoOohWU28Y_WXoOohWU28Y
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000001.tar
Processed videos/TV-DjM8242s/TV-DjM8242s -> videos_TV-DjM8242s_TV-DjM8242s
Processed videos/rCVlIVKqqGE/rCVlIVKqqGE -> videos_rCVlIVKqqGE_rCVlIVKqqGE
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000002.tar
Processed videos/lb_5AdUpfuA/lb_5AdUpfuA -> videos_lb_5AdUpfuA_lb_5AdUpfuA
Processed videos/FCQ-rih6cHY/FCQ-rih6cHY -> videos_FCQ-rih6cHY_FCQ-rih6cHY
Creating C:\Users\sahil\Downloads\AIVideos\webdataset\shard-000003.tar
Processed videos/eQ6UE968Xe4/eQ6UE968Xe4 -> videos_eQ6UE968Xe4_eQ6UE968Xe4
Processed videos/eFgkZKhNUdM/eFgkZKhNUdM -> videos_eFgkZKhNUdM_eFgkZKhNUdM
Shards created in C:\Users\sahil\Downloads\AIVideos\webdataset


### 🔁 Streaming Sharded Video Data with WebDataset

Once the dataset is packaged into `.tar` shards, we use [WebDataset](https://github.com/webdataset/webdataset) to stream data directly from disk—avoiding memory overhead from fully unpacked videos. On Windows, path formatting can interfere with URI resolution, so we construct proper `file://` URIs using `pathlib`.

The dataset is loaded lazily using the `decode()` method, allowing frame-by-frame or sample-by-sample access without disk I/O bottlenecks. This snippet verifies correct URI formatting, initializes the dataset, and checks that samples are accessible—an essential step before large-scale processing or training.


In [ ]:
import webdataset as wds
from pathlib import Path

def get_windows_uris(shard_dir):
    """Get properly formatted Windows URIs for WebDataset"""
    base_path = Path(shard_dir)
    shard_files = sorted(base_path.glob("shard-*.tar"))
    
    uris = []
    for f in shard_files:
        # Use pathlib's built-in URI conversion
        uri = f.resolve().as_uri()
        # Fix double slashes in Windows paths
        uri = uri.replace("file:///", "file://")
        uris.append(uri)
    
    return uris

# Get properly formatted URIs
uris = get_windows_uris(r"C:\Users\sahil\Downloads\AIVideos\webdataset")

# Verify URI formatting
print("Sample URIs:")
for uri in uris[:3]:
    print(uri)  # Should look like: file://C:/Users/sahil/.../shard-000000.tar

# Create dataset
dataset = wds.WebDataset(uris).decode()

# Test access
for sample in dataset:
    key = sample["__key__"]
    print(f"Successfully accessed sample: {key}")
    print(f"MP4 size: {len(sample['mp4'])} bytes")
      # Test first sample only

Sample URIs:
file://C:/Users/sahil/Downloads/AIVideos/webdataset/shard-000000.tar
file://C:/Users/sahil/Downloads/AIVideos/webdataset/shard-000001.tar
file://C:/Users/sahil/Downloads/AIVideos/webdataset/shard-000002.tar
Successfully accessed sample: videos_9CGGh6ivg68_9CGGh6ivg68
MP4 size: 56171526 bytes
Successfully accessed sample: videos_WXoOohWU28Y_WXoOohWU28Y
MP4 size: 126281096 bytes
Successfully accessed sample: videos_TV-DjM8242s_TV-DjM8242s
MP4 size: 342311506 bytes
Successfully accessed sample: videos_rCVlIVKqqGE_rCVlIVKqqGE
MP4 size: 129960385 bytes
Successfully accessed sample: videos_lb_5AdUpfuA_lb_5AdUpfuA
MP4 size: 110658026 bytes
Successfully accessed sample: videos_FCQ-rih6cHY_FCQ-rih6cHY
MP4 size: 342702373 bytes
Successfully accessed sample: videos_eQ6UE968Xe4_eQ6UE968Xe4
MP4 size: 630914754 bytes
Successfully accessed sample: videos_eFgkZKhNUdM_eFgkZKhNUdM
MP4 size: 358151645 bytes


In [ ]:
!pip install opencv-python-headless webdataset pymongo tqdm scikit-image Pillow --user

  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scikit_image-0.25.2-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.37.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
Using cached scikit_image-0.25.2-cp313-cp313-win_amd64.whl (12.9 MB)
Using cached imageio-2.37.0-py3-none-any.whl (315 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)


### 🎞️ Extracting Subtitle-Aligned Frames with WebDataset

To enable fine-grained video understanding, we extract a representative frame from each subtitle segment in the lecture videos. This pipeline leverages `WebDataset` to stream `.tar` shards and decode video, subtitle, and metadata components on the fly.

Each `.mp4` is decoded frame-by-frame using OpenCV. For every subtitle, we compute its midpoint timestamp and extract the corresponding frame. A similarity check (based on correlation of downscaled grayscale versions) removes near-duplicate frames across subtitle segments to avoid redundancy.

All resulting frames are saved as `.jpg` files in a structured directory layout for downstream tasks such as retrieval, clustering, or question generation. A final verification pass ensures completeness and consistency across the dataset.

This step prepares the dataset for effective visual-semantic indexing and retrieval.


In [ ]:
import os
import webdataset as wds
from pathlib import Path
import json
import tempfile
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import logging
import re
from tqdm import tqdm  
from bson import Binary


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class VideoProcessor:
    def __init__(self, output_dir):
        self.similarity_threshold = 0.95
        self.num_threads = min(32, (os.cpu_count() or 1) + 4)  # Auto-configured threads
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

    def process_shards(self, data_dir):
        """Process WebDataset shards with multithreaded decoding"""
        data_path = Path(data_dir)
        shards = sorted(data_path.glob("shard-*.tar"))
        
        # Create dataset with path conversion for Kaggle (ensure file paths are correct)
        dataset = wds.WebDataset([
            f"file://{shard.resolve().as_posix()}" for shard in shards
        ]).decode("rgb").map(self._decode_sample)
        
        # Use tqdm for progress bar (Kaggle compatible)
        with tqdm(total=len(shards), desc="Processing videos") as pbar:
            for sample in dataset:
                try:
                    self._process_sample(sample)
                    pbar.update(1)  # Update progress bar for each processed sample
                except Exception as e:
                    logger.error(f"Failed processing sample: {str(e)}")

    def _decode_sample(self, sample):
        """Decode JSON and VTT data properly"""
        if "json" in sample and isinstance(sample["json"], bytes):
            sample["json"] = json.loads(sample["json"].decode("utf-8"))
        if "vtt" in sample and isinstance(sample["vtt"], bytes):
            sample["vtt"] = sample["vtt"].decode("utf-8")
        return sample

    def _process_sample(self, sample):
        """Process individual sample with frame-subtitle alignment"""
        key = sample["__key__"]
        video_bytes = sample["mp4"]
        subtitles = self._parse_vtt(sample["vtt"])
        metadata = sample["json"]

        # Process video frames
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
            temp_file.write(video_bytes)
            temp_file.flush()
            frames = self._extract_aligned_frames(temp_file.name, subtitles, key)

        if frames:
            logger.info(f"Processed {len(frames)} frames for {key}")
            self._save_frames(frames, key)

    def _extract_aligned_frames(self, video_path, subtitles, key):
        """Extract and deduplicate frames aligned with subtitles"""
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return []

        fps = cap.get(cv2.CAP_PROP_FPS)
        frames = []
        prev_frame = None

        try:
            for sub in subtitles:
                start = self._vtt_to_seconds(sub["start_time"])
                end = self._vtt_to_seconds(sub["end_time"])
                target_time = start + (end - start) / 2
                frame_pos = int(target_time * fps)
                
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
                ret, frame = cap.read()
                
                if not ret:
                    continue

                if prev_frame is not None and self._is_similar(prev_frame, frame):
                    continue

                frames.append(frame)
                prev_frame = frame.copy()

        finally:
            cap.release()

        return frames

    def _is_similar(self, frame1, frame2):
        """Check frame similarity using downscaled SSIM"""
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray1 = cv2.resize(gray1, (224, 224))
        gray2 = cv2.resize(gray2, (224, 224))
        score = np.corrcoef(gray1.flatten(), gray2.flatten())[0, 1]
        return score > self.similarity_threshold

    def _save_frames(self, frames, key):
        """Save frames to output directory"""
        frame_dir = os.path.join(self.output_dir, key)
        os.makedirs(frame_dir, exist_ok=True)

        for idx, frame in enumerate(frames):
            frame_path = os.path.join(frame_dir, f"frame_{idx:04d}.jpg")
            frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_image.save(frame_path)
        logger.info(f"Frames saved for {key} to {frame_dir}")

    def _verify_frames(self):
        """Verify that all frames have been saved"""
        missing_frames = []
        for subdir, dirs, files in os.walk(self.output_dir):
            for dir in dirs:
                frame_files = sorted(Path(subdir).joinpath(dir).glob("frame_*.jpg"))
                if len(frame_files) == 0:
                    missing_frames.append(dir)
        if missing_frames:
            logger.error(f"Missing frames for videos: {', '.join(missing_frames)}")
        else:
            logger.info("All frames are extracted and stored successfully.")

    @staticmethod
    def _parse_vtt(content):
        """Parse VTT subtitles into time segments"""
        subtitles = []
        pattern = re.compile(r'(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})')
        for line in content.split('\n'):
            match = pattern.search(line)
            if match:
                start, end = match.groups()
                text = line.split('\n')[-1].strip()
                if text:
                    subtitles.append({
                        "start_time": start,
                        "end_time": end,
                        "text": text
                    })
        return subtitles

    @staticmethod
    def _vtt_to_seconds(time_str):
        """Convert VTT timestamp to seconds"""
        parts = time_str.replace(',', '.').split(':')
        if len(parts) == 3:
            return int(parts[0])*3600 + int(parts[1])*60 + float(parts[2])
        elif len(parts) == 2:
            return int(parts[0])*60 + float(parts[1])
        return 0.0

if __name__ == "__main__":
    output_dir = r"C:\Users\sahil\Downloads\AIVideos\processed_data"  # Set your desired output directory
    processor = VideoProcessor(output_dir)
    
    # Replace '/path/to/shards' with the actual directory containing your dataset shards
    dataset_dir = r"C:\Users\sahil\Downloads\AIVideos\webdataset"
    processor.process_shards(dataset_dir)

    # Verify if frames are properly extracted and stored
    processor._verify_frames()
    logger.info("Processing and verification completed successfully")


2025-04-01 17:53:41,296 - INFO - Processed 18 frames for videos_9CGGh6ivg68_9CGGh6ivg68
2025-04-01 17:53:41,464 - INFO - Frames saved for videos_9CGGh6ivg68_9CGGh6ivg68 to C:\Users\sahil\Downloads\AIVideos\processed_data\videos_9CGGh6ivg68_9CGGh6ivg68
c:\Users\sahil\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:2999: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\sahil\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3000: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
2025-04-01 17:54:08,814 - INFO - Processed 109 frames for videos_WXoOohWU28Y_WXoOohWU28Y
2025-04-01 17:54:09,612 - INFO - Frames saved for videos_WXoOohWU28Y_WXoOohWU28Y to C:\Users\sahil\Downloads\AIVideos\processed_data\videos_WXoOohWU28Y_WXoOohWU28Y
2025-04-01 17:55:38,381 - INFO - Processed 265 frames for videos_TV-DjM8242s_TV-DjM8242s
2025-04-01 17:55:40,345 - INFO -

In [90]:
import os
import webdataset as wds
from pathlib import Path
import json
import tempfile
import cv2
import numpy as np
from PIL import Image
from io import BytesIO
import logging
import re
from tqdm import tqdm  
import base64


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class VideoProcessor:
    def __init__(self, output_dir):
        self.similarity_threshold = 0.95
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

    def process_shards(self, data_dir):
        """Process WebDataset shards with multithreaded decoding"""
        data_path = Path(data_dir)
        shards = sorted(data_path.glob("shard-*.tar"))
        
        if not shards:
            logger.error(f"No shards found in directory: {data_dir}")
            return
        
        # Create dataset with path conversion for Windows (ensure file paths are correct)
        dataset = wds.WebDataset([ 
            f"file://{shard.resolve().as_posix()}" for shard in shards
        ]).decode("rgb").map(self._decode_sample)
        
        # Use tqdm for progress bar with fixed length
        with tqdm(total=len(shards), desc="Processing videos") as pbar:
            shard_count = 0
            for sample in dataset:
                try:
                    self._process_sample(sample)
                    # Update progress after each sample - not ideal but WebDataset doesn't expose sample count per shard
                    # This is a simplification - in reality you'd need a counter per shard
                    pbar.n = min(shard_count + 1, len(shards))
                    pbar.refresh()
                except Exception as e:
                    logger.error(f"Failed processing sample: {str(e)}")
                shard_count += 1
            
            # Ensure progress bar reaches 100% at the end
            pbar.n = len(shards)
            pbar.refresh()

    def _decode_sample(self, sample):
        """Decode JSON and VTT data properly"""
        if "json" in sample and isinstance(sample["json"], bytes):
            try:
                sample["json"] = json.loads(sample["json"].decode("utf-8"))
            except json.JSONDecodeError as e:
                logger.error(f"Error decoding JSON: {str(e)}")
                sample["json"] = {}
                
        if "vtt" in sample and isinstance(sample["vtt"], bytes):
            sample["vtt"] = sample["vtt"].decode("utf-8")
        return sample

    def _process_sample(self, sample):
        """Process individual sample with frame-subtitle alignment"""
        # Check if sample has all required fields
        required_fields = ["__key__", "mp4", "vtt"]
        for field in required_fields:
            if field not in sample:
                logger.error(f"Missing required field: {field} in sample")
                return
                
        key = sample["__key__"]
        video_bytes = sample["mp4"]
        subtitles = self._parse_vtt(sample["vtt"])
        metadata = sample.get("json", {})  # Use empty dict if json is not present

        # Process video frames
        with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as temp_file:
            temp_path = temp_file.name
            temp_file.write(video_bytes)
            temp_file.flush()
            
        try:
            frames = self._extract_aligned_frames(temp_path, subtitles, key)

            if frames:
                logger.info(f"Processed {len(frames)} frames for {key}")
                self._prepare_for_upload(frames, key, subtitles)
        except Exception as e:
            logger.error(f"Error processing video {key}: {str(e)}")
        finally:
            # Clean up the temporary file
            try:
                os.unlink(temp_path)
            except Exception as e:
                logger.warning(f"Failed to delete temporary file {temp_path}: {str(e)}")

    def _extract_aligned_frames(self, video_path, subtitles, key):
        """Extract and deduplicate frames aligned with subtitles"""
        if not subtitles:
            logger.warning(f"No subtitles found for {key}")
            return []
            
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logger.error(f"Error opening video file: {video_path}")
            return []

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:
            logger.warning(f"Invalid FPS value ({fps}) for {key}, using default of 25")
            fps = 25
            
        frames = []
        prev_frame = None

        try:
            for sub in subtitles:
                start = self._vtt_to_seconds(sub["start_time"])
                end = self._vtt_to_seconds(sub["end_time"])
                
                # Validate timestamp values
                if start < 0 or end < 0 or start >= end:
                    logger.warning(f"Invalid subtitle timing: {sub}")
                    continue
                    
                target_time = start + (end - start) / 2
                frame_pos = int(target_time * fps)
                
                # Seek to the frame
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
                ret, frame = cap.read()
                
                if not ret:
                    logger.warning(f"Failed to read frame at position {frame_pos} for {key}")
                    continue

                if prev_frame is not None and self._is_similar(prev_frame, frame):
                    continue

                frames.append(frame)
                prev_frame = frame.copy()

        finally:
            cap.release()

        return frames

    def _is_similar(self, frame1, frame2):
        """Check frame similarity using downscaled SSIM"""
        try:
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            gray1 = cv2.resize(gray1, (224, 224))
            gray2 = cv2.resize(gray2, (224, 224))
            score = np.corrcoef(gray1.flatten(), gray2.flatten())[0, 1]
            return score > self.similarity_threshold
        except Exception as e:
            logger.warning(f"Error comparing frames: {str(e)}")
            return False

    def _prepare_for_upload(self, frames, key, subtitles):
        """Prepare frames and metadata for upload"""
        frame_data = []
        
        for idx, frame in enumerate(frames):
            try:
                # Convert frame to base64 for JSON compatibility
                frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                byte_io = BytesIO()
                frame_image.save(byte_io, format="JPEG", quality=85)  # Reduced quality for smaller file size
                byte_io.seek(0)
                frame_binary = byte_io.getvalue()

                # Convert Binary to Base64 (JSON serializable)
                frame_base64 = base64.b64encode(frame_binary).decode('utf-8')

                # Prepare metadata
                frame_metadata = {
                    "video_key": key,
                    "frame_idx": idx,
                    "frame_base64": frame_base64,  # Store the frame as a base64 string
                    "subtitles": [sub for sub in subtitles if sub.get("text")]  # Only store non-empty subtitles
                }
                frame_data.append(frame_metadata)
            except Exception as e:
                logger.error(f"Error preparing frame {idx} for {key}: {str(e)}")

        if not frame_data:
            logger.warning(f"No valid frames prepared for {key}")
            return
            
        # Fix the path issue: ensure directory structure exists and normalize path
        try:
            # Create a safe filename by replacing path separators
            safe_key = key.replace('/', '_').replace('\\', '_')
            frame_data_file = os.path.join(self.output_dir, f"{safe_key}_frames.json")
            
            # Ensure the output directory exists
            os.makedirs(os.path.dirname(os.path.abspath(frame_data_file)), exist_ok=True)
            
            with open(frame_data_file, "w") as f:
                json.dump(frame_data, f)
            logger.info(f"Stored frame data for {key} in file: {frame_data_file}")
        except Exception as e:
            logger.error(f"Error saving frame data for {key}: {str(e)}")

    def _verify_frames(self):
        """Verify that all frames have been saved"""
        try:
            frame_files = list(Path(self.output_dir).glob("*_frames.json"))
            if not frame_files:
                logger.warning(f"No frame files found in {self.output_dir}")
                return
                
            logger.info(f"Found {len(frame_files)} frame files in {self.output_dir}")
            
            # Check file validity
            invalid_files = []
            for file_path in frame_files:
                try:
                    with open(file_path, "r") as f:
                        data = json.load(f)
                    if not data:
                        invalid_files.append(file_path.name)
                except Exception:
                    invalid_files.append(file_path.name)
            
            if invalid_files:
                logger.error(f"Found {len(invalid_files)} invalid frame files: {', '.join(invalid_files[:5])}" +
                           (f" and {len(invalid_files) - 5} more" if len(invalid_files) > 5 else ""))
            else:
                logger.info("All frame files are valid.")
        except Exception as e:
            logger.error(f"Error verifying frames: {str(e)}")

    @staticmethod
    def _parse_vtt(content):
        """Parse VTT subtitles into time segments and clean up unnecessary styling/positioning info"""
        if not content or not isinstance(content, str):
            return []
            
        subtitles = []
        pattern = re.compile(r'(\d{2}:\d{2}:\d{2}\.\d{3}) --> (\d{2}:\d{2}:\d{2}\.\d{3})')
        
        # Split content into lines and process each line
        lines = content.split('\n')
        subtitle = {}
        subtitle_text = []
        
        for line in lines:
            match = pattern.search(line)
            if match:
                # If there's already a subtitle in progress, store it
                if subtitle and subtitle_text:
                    subtitle["text"] = ' '.join(subtitle_text).strip()  # Combine all subtitle lines
                    subtitles.append(subtitle)
                    subtitle = {}  # Reset for the next subtitle
                    subtitle_text = []  # Reset text collection for next subtitle
                
                # Extract start and end times
                start, end = match.groups()
                subtitle = {"start_time": start, "end_time": end}
            
            elif line.strip() and subtitle:  # If the line is not empty and we have a subtitle in progress
                # Clean up the subtitle text (remove align, position, and extra spaces)
                cleaned_text = re.sub(r'(\s+align:\S+|\s+position:\S+)', '', line.strip())
                
                # Collect all parts of the subtitle text
                subtitle_text.append(cleaned_text)

        # Add the last subtitle if present
        if subtitle and subtitle_text:
            subtitle["text"] = ' '.join(subtitle_text).strip()  # Combine all subtitle lines
            subtitles.append(subtitle)
        
        return subtitles

    @staticmethod
    def _vtt_to_seconds(time_str):
        """Convert VTT timestamp to seconds"""
        if not time_str:
            return 0.0
            
        try:
            parts = time_str.replace(',', '.').split(':')
            if len(parts) == 3:
                return int(parts[0])*3600 + int(parts[1])*60 + float(parts[2])
            elif len(parts) == 2:
                return int(parts[0])*60 + float(parts[1])
        except (ValueError, IndexError) as e:
            logger.warning(f"Error parsing timestamp {time_str}: {str(e)}")
        return 0.0

if __name__ == "__main__":
    # Use try-except for main error handling
    try:
        output_dir = r"C:\Users\sahil\Downloads\AIVideos\processed_data"  # Set your desired output directory
        processor = VideoProcessor(output_dir)
        
        # Replace '/path/to/shards' with the actual directory containing your dataset shards
        dataset_dir = r"C:\Users\sahil\Downloads\AIVideos\webdataset"
        processor.process_shards(dataset_dir)

        # Verify if frames are properly extracted
        processor._verify_frames()
        logger.info("Processing and verification completed successfully")
    except Exception as e:
        logger.critical(f"Critical error in main execution: {str(e)}")

Processing videos:   0%|          | 0/4 [00:00<?, ?it/s]2025-04-03 12:00:31,045 - WARNING - Failed to read frame at position 8565 for videos/9CGGh6ivg68/9CGGh6ivg68
2025-04-03 12:00:31,060 - INFO - Processed 18 frames for videos/9CGGh6ivg68/9CGGh6ivg68
2025-04-03 12:00:31,273 - INFO - Stored frame data for videos/9CGGh6ivg68/9CGGh6ivg68 in file: C:\Users\sahil\Downloads\AIVideos\processed_data\videos_9CGGh6ivg68_9CGGh6ivg68_frames.json
Processing videos:  25%|██▌       | 1/4 [00:12<00:37, 12.60s/it]2025-04-03 12:01:03,605 - INFO - Processed 108 frames for videos/WXoOohWU28Y/WXoOohWU28Y
2025-04-03 12:01:04,919 - INFO - Stored frame data for videos/WXoOohWU28Y/WXoOohWU28Y in file: C:\Users\sahil\Downloads\AIVideos\processed_data\videos_WXoOohWU28Y_WXoOohWU28Y_frames.json
Processing videos:  50%|█████     | 2/4 [00:46<00:46, 23.14s/it]2025-04-03 12:02:32,276 - INFO - Processed 260 frames for videos/TV-DjM8242s/TV-DjM8242s
2025-04-03 12:02:36,186 - INFO - Stored frame data for videos/TV-Dj

### 🗄️ Connecting to MongoDB for Frame and Metadata Storage

This section initializes a connection to a remote MongoDB instance using the `pymongo` library. We connect to the `AIframes` database and select the `AIprocessedframes` collection, which will store structured frame data along with associated metadata (e.g., subtitle segments, timestamps, and video identifiers).

Establishing a secure connection also validates the availability of the remote database by listing existing database names. This enables us to integrate extracted visual-semantic data into a persistent, queryable format for downstream retrieval and analysis.


In [ ]:
from pymongo import MongoClient
import json
import os
import base64
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


MONGO_URI = "MASKED"
try:
    # Establish connection
    client = MongoClient(MONGO_URI)
    
    # List available databases (test connection)
    db_list = client.list_database_names()
    print("Connected to MongoDB successfully!")
    print("Databases:", db_list)

except Exception as e:
    print("Connection failed:", e)

db = client["AIframes"]
collection = db["AIprocessedframes"]

Connected to MongoDB successfully!
Databases: ['AIframes', 'admin', 'local']


### 🧠 Storing Extracted Frame Metadata in MongoDB

This block handles the ingestion of frame-level metadata and image paths into a MongoDB collection. After frames are extracted and serialized into JSON files locally, this function:

1. **Reads all `_frames.json` files** from a specified directory.
2. **Parses and uploads** the structured metadata (including timestamps, frame paths, and associated video keys) into the `AIprocessedframes` collection.
3. **Logs the total number of frames** uploaded per video for verification.

A verification routine follows to ensure each video has its frames successfully recorded in the database. This ensures downstream processes like embedding, search, or video slicing can rely on persistent, indexed access to visual segments.


In [ ]:


def store_frames_in_mongo(processed_dir):
    """Store extracted frames and metadata from local JSON files into MongoDB."""
    json_files = [f for f in os.listdir(processed_dir) if f.endswith("_frames.json")]

    if not json_files:
        logger.warning("No JSON files found in the directory. Ensure frames are extracted.")
        return

    for json_file in json_files:
        file_path = os.path.join(processed_dir, json_file)
        with open(file_path, "r") as f:
            frame_data = json.load(f)

        if frame_data:
            collection.insert_many(frame_data)
            logger.info(f"Uploaded {len(frame_data)} frames from {json_file} to MongoDB.")

def verify_mongo_storage():
    """Verify that all frames are correctly stored in MongoDB."""
    stored_videos = collection.distinct("video_key")
    logger.info(f"Stored videos in MongoDB: {len(stored_videos)}")

    for video_key in stored_videos:
        frame_count = collection.count_documents({"video_key": video_key})
        logger.info(f"Video {video_key}: {frame_count} frames stored.")

if __name__ == "__main__":
    processed_dir = r"C:\Users\sahil\Downloads\AIVideos\processed_data"  # Directory containing processed JSON files
    store_frames_in_mongo(processed_dir)
    verify_mongo_storage()
    logger.info("MongoDB storage and verification completed successfully.")


2025-04-02 10:45:23,165 - INFO - Uploaded 18 frames from videos_9CGGh6ivg68_9CGGh6ivg68_frames.json to MongoDB.
2025-04-02 10:45:37,287 - INFO - Uploaded 409 frames from videos_eFgkZKhNUdM_eFgkZKhNUdM_frames.json to MongoDB.
2025-04-02 10:46:13,281 - INFO - Uploaded 756 frames from videos_eQ6UE968Xe4_eQ6UE968Xe4_frames.json to MongoDB.
2025-04-02 10:46:24,254 - INFO - Uploaded 296 frames from videos_FCQ-rih6cHY_FCQ-rih6cHY_frames.json to MongoDB.


OperationFailure: you are over your space quota, using 558 MB of 512 MB, full error: {'ok': 0, 'errmsg': 'you are over your space quota, using 558 MB of 512 MB', 'code': 8000, 'codeName': 'AtlasError'}

### ⚠️ MongoDB Storage Quota Error

While attempting to upload the extracted frame metadata into MongoDB Atlas, an `OperationFailure` occurred due to exceeding the allocated storage quota. The current usage was **558 MB**, while the limit is **512 MB** for the free tier. This indicates that either the dataset is too large for the current tier or that older data needs to be cleaned to free up space. Resolving this may involve upgrading to a paid plan, deleting unused collections, or using a local MongoDB instance for continued experimentation.


### 🧹 Enhanced MongoDB Upload with Cleanup & Size Monitoring

To prevent re-uploading redundant data and to manage MongoDB quota limits efficiently, the upload script now includes several improvements:

- **Selective Deletion**: You can delete all data or only the data related to a specific `video_key` before uploading new frames. This prevents duplication and helps with quota management.
- **File Size Logging**: The script logs the size of each JSON file being uploaded, allowing you to monitor space usage proactively.
- **Robust Error Handling**: Any insertion issues are caught and logged without stopping the process.
- **Verification Step**: After upload, the script prints a summary of how many frames are stored per video in MongoDB.

These changes ensure smoother interaction with MongoDB Atlas, especially when working within the storage constraints of the free tier.


In [91]:
from pymongo import MongoClient
import json
import os
import base64
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


MONGO_URI = "mongodb://localhost:27017"

try:
    # Establish connection
    client = MongoClient(MONGO_URI)
    
    # List available databases (test connection)
    db_list = client.list_database_names()
    print("Connected to MongoDB successfully!")
    print("Databases:", db_list)

except Exception as e:
    print("Connection failed:", e)

db = client["AIframes"]
collection = db["AIprocessedframes"]

Connected to MongoDB successfully!
Databases: ['AIframes', 'admin', 'config', 'local', 'video_db', 'video_rag']


In [ ]:
import os
import json
import io
import base64
import logging
from PIL import Image
from pymongo import MongoClient

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Update if using remote DB
db = client["video_rag"]  # Replace with your database name
collection = db["video_frames"]  # Replace with your collection name

def get_file_size_in_mb(file_path):
    """Returns the size of the file in MB."""
    return os.path.getsize(file_path) / (1024 * 1024)  # Convert bytes to MB

def compress_image(image_path, quality=50):
    """Compress an image using JPEG format and return base64-encoded compressed data."""
    with Image.open(image_path) as img:
        img = img.convert("RGB")  # Ensure it's in RGB mode
        img_io = io.BytesIO()
        img.save(img_io, format="JPEG", quality=quality)  # Adjust quality for compression
        compressed_data = img_io.getvalue()
        return base64.b64encode(compressed_data).decode("utf-8")  # Convert to base64

def delete_previous_data(video_key=None):
    """Delete previously uploaded data from the collection."""
    if video_key:
        logger.info(f"Deleting previous data for video_key: {video_key}")
        result = collection.delete_many({"video_key": video_key})
    else:
        logger.info("Deleting all previously uploaded data.")
        result = collection.delete_many({})
    logger.info(f"Deleted {result.deleted_count} documents.")

def store_frames_in_mongo(processed_dir, video_key=None):
    """Store frames with their corresponding subtitles in MongoDB."""
    json_files = [f for f in os.listdir(processed_dir) if f.endswith("_frames.json")]
    vtt_files = {f.split(".")[0]: os.path.join(processed_dir, f) for f in os.listdir(processed_dir) if f.endswith(".vtt")}

    if not json_files:
        logger.warning("No JSON files found in the directory. Ensure frames are extracted.")
        return

    delete_previous_data(video_key)  # Clear previous data

    for json_file in json_files:
        file_path = os.path.join(processed_dir, json_file)
        logger.info(f"Processing {json_file}")

        with open(file_path, "r") as f:
            frame_data = json.load(f)

        # Get subtitles for this video
        video_id = json_file.split("_frames.json")[0]  # Extract video identifier
        subtitles = extract_subtitles(vtt_files.get(video_id, "")) if video_id in vtt_files else []

        for frame in frame_data:
            # Convert image to base64
            if "frame_path" in frame:
                frame["frame_base64"] = compress_image(frame["frame_path"])
                del frame["frame_path"]

            # Attach subtitles
            frame["subtitles"] = subtitles  # Store the full list of subtitles

        if frame_data:
            try:
                collection.insert_many(frame_data)
                logger.info(f"Uploaded {len(frame_data)} frames from {json_file} to MongoDB.")
            except Exception as e:
                logger.error(f"Error inserting frames from {json_file} to MongoDB: {e}")


    for json_file in json_files:
        file_path = os.path.join(processed_dir, json_file)
        original_size = get_file_size_in_mb(file_path)
        
        logger.info(f"Processing {json_file} (Size: {original_size:.2f} MB)")
        
        with open(file_path, "r") as f:
            frame_data = json.load(f)
        
        for frame in frame_data:
            if "frame_path" in frame:
                frame["frame_base64"] = compress_image(frame["frame_path"])  # Compress before storing
                del frame["frame_path"]  # Remove original path to save space
            
            # Handle subtitles correctly
            if "subtitles" in frame and isinstance(frame["subtitles"], list):
                frame["subtitles"] = " ".join([s["text"] for s in frame["subtitles"]])  # Extract text
            
        compressed_size = sum(len(json.dumps(doc).encode('utf-8')) for doc in frame_data) / (1024 * 1024)
        logger.info(f"Compressed data size for {json_file}: {compressed_size:.2f} MB")
        
        if frame_data:
            try:
                collection.insert_many(frame_data)
                logger.info(f"Uploaded {len(frame_data)} compressed frames from {json_file} to MongoDB.")
            except Exception as e:
                logger.error(f"Error inserting frames from {json_file} to MongoDB: {e}")

def verify_mongo_storage():
    """Verify that all frames are correctly stored in MongoDB."""
    stored_videos = collection.distinct("video_key")
    logger.info(f"Stored videos in MongoDB: {len(stored_videos)}")

    for video_key in stored_videos:
        frame_count = collection.count_documents({"video_key": video_key})
        logger.info(f"Video {video_key}: {frame_count} frames stored.")

if __name__ == "__main__":
    #processed_dir = os.path.join("C:", "Users", "sahil", "Downloads", "AIVideos", "processed_data") 
    processed_dir = r"C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data" 
    store_frames_in_mongo(processed_dir)
    verify_mongo_storage()
    logger.info("MongoDB storage and verification completed successfully.")


2025-04-03 10:08:04,172 - INFO - Deleting all previously uploaded data.
2025-04-03 10:08:06,109 - INFO - Deleted 3954 documents.
2025-04-03 10:08:06,110 - INFO - Processing videos_9CGGh6ivg68_9CGGh6ivg68_frames.json
2025-04-03 10:08:06,152 - INFO - Uploaded 18 frames from videos_9CGGh6ivg68_9CGGh6ivg68_frames.json to MongoDB.
2025-04-03 10:08:06,153 - INFO - Processing videos_eFgkZKhNUdM_eFgkZKhNUdM_frames.json
2025-04-03 10:08:07,143 - INFO - Uploaded 409 frames from videos_eFgkZKhNUdM_eFgkZKhNUdM_frames.json to MongoDB.
2025-04-03 10:08:07,144 - INFO - Processing videos_eQ6UE968Xe4_eQ6UE968Xe4_frames.json
2025-04-03 10:08:09,699 - INFO - Uploaded 756 frames from videos_eQ6UE968Xe4_eQ6UE968Xe4_frames.json to MongoDB.
2025-04-03 10:08:09,700 - INFO - Processing videos_FCQ-rih6cHY_FCQ-rih6cHY_frames.json
2025-04-03 10:08:10,765 - INFO - Uploaded 296 frames from videos_FCQ-rih6cHY_FCQ-rih6cHY_frames.json to MongoDB.
2025-04-03 10:08:10,766 - INFO - Processing videos_lb_5AdUpfuA_lb_5AdUpf

### Subtitle Extraction and Storage Strategy

This project involves extracting subtitles from `.vtt` files and storing them for downstream retrieval tasks. Two strategies were considered:

---

#### Option 1: Save Subtitles as `.txt` Files

- Each video's subtitle file is saved in a separate directory.
- Good for manual inspection or traditional media workflows.
- Not ideal for integration with machine learning pipelines.

---

#### Option 2: Store Subtitles in MongoDB (**Chosen**)

- Subtitles are parsed and stored as structured documents in a MongoDB collection.
- Each document contains a `video_key` and a list of subtitle segments (with start, end, and text).
- Includes upsert logic to avoid duplicate entries.
- Ideal for:
  - Retrieval-Augmented Generation (RAG)
  - Frame-to-subtitle alignment
  - Embedding pipelines
  - Querying subtitles dynamically

---

#### Why Option 2 Is Better

✅ Centralized data storage with other video metadata  
✅ Supports programmatic querying and integration with AI tools  
✅ Scalable and consistent with the rest of the video processing pipeline  

> If needed, `.txt` export can be added later for portability or manual review.


In [65]:
import os
import webvtt
import logging
from pymongo import MongoClient

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Connect to MongoDB (Update the URI if using a remote database)
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]  # Replace with your database name
collection = db["subtitles"]  # Replace with your collection name

def extract_subtitles(vtt_file):
    """Extract subtitles from a WebVTT (.vtt) file and log the extraction process."""
    subtitles = []
    try:
        # Read and extract captions from the VTT file
        for caption in webvtt.read(vtt_file):
            subtitle = {
                "start": caption.start,
                "end": caption.end,
                "text": caption.text
            }
            subtitles.append(subtitle)
            
            # Log each subtitle being extracted
            logger.info(f"Extracted subtitle: {subtitle['start']} --> {subtitle['end']} | {subtitle['text']}")
        
        # If no subtitles are extracted, log a warning
        if not subtitles:
            logger.warning(f"No subtitles found in {vtt_file}")
        
    except Exception as e:
        logger.error(f"Error extracting subtitles from {vtt_file}: {e}")

    return subtitles  # Return the list of subtitle segments

def extract_and_store_subtitles_to_mongodb(dataset_dir):
    """Extract subtitles from all videos in the dataset and store in MongoDB."""
    # Iterate through the dataset directory to find .vtt subtitle files, including subdirectories
    video_files = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(".vtt"):
                video_files.append(os.path.join(root, file))

    if not video_files:
        logger.warning("No .vtt files found in the dataset directory.")
        return

    for vtt_file in video_files:
        # Extract subtitles from the .vtt file
        subtitles = extract_subtitles(vtt_file)

        # Prepare the video key (use the base name of the .vtt file as the key)
        video_key = os.path.splitext(os.path.basename(vtt_file))[0]

        # Prepare data to be stored in MongoDB
        video_data = {
            "video_key": video_key,
            "subtitles": subtitles
        }
        
        # Store subtitles in MongoDB
        try:
            # Upsert: If video already exists, update it; otherwise, insert a new document
            collection.update_one(
                {"video_key": video_key},  # Query to check if the document exists
                {"$set": video_data},  # Update with the new subtitles data
                upsert=True  # Create a new document if it doesn't exist
            )
            logger.info(f"Subtitles for {video_key} uploaded to MongoDB.")
        except Exception as e:
            logger.error(f"Error uploading subtitles for {video_key} to MongoDB: {e}")

if __name__ == "__main__":
    dataset_dir = r"C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data"  # Replace with the path to your dataset
    extract_and_store_subtitles_to_mongodb(dataset_dir)
    logger.info("Subtitle extraction and storage to MongoDB completed successfully.")


2025-04-03 10:25:30,272 - INFO - Extracted subtitle: 00:00:02.990 --> 00:00:03.000 | in this video I would like to start the
2025-04-03 10:25:30,274 - INFO - Extracted subtitle: 00:00:03.000 --> 00:00:04.870 | in this video I would like to start the
discussion about convolutional new
2025-04-03 10:25:30,275 - INFO - Extracted subtitle: 00:00:04.870 --> 00:00:04.880 | discussion about convolutional new
2025-04-03 10:25:30,276 - INFO - Extracted subtitle: 00:00:04.880 --> 00:00:07.389 | discussion about convolutional new
networks which is another architecture
2025-04-03 10:25:30,276 - INFO - Extracted subtitle: 00:00:07.389 --> 00:00:07.399 | networks which is another architecture
2025-04-03 10:25:30,277 - INFO - Extracted subtitle: 00:00:07.399 --> 00:00:09.830 | networks which is another architecture
of uh neural networks that we are going
2025-04-03 10:25:30,277 - INFO - Extracted subtitle: 00:00:09.830 --> 00:00:09.840 | of uh neural networks that we are going
2025-04-03 10:25:30,278

### 🎯 Subtitle-Based Question Generation

This module implements a basic question generation pipeline that creates questions from subtitles aligned with video frames. It is designed to process frame metadata and subtitles (stored in JSON files) and output a structured dataset of question-answer pairs, with references to aligned frames for possible use in training a vision-language model.

---

#### ⚙️ Functionality Overview

- **Input**: Extracted frame JSON files with base64-encoded frames and subtitles.
- **Output**: 
  - A `.json` file containing generated questions and metadata (`subtitle_questions.json`).
  - A directory of individual `.txt` files, each storing a base64-encoded frame, named by `video_key` and `frame_idx`.

- **Workflow**:
  1. Iterates over subtitle-aligned frame JSONs.
  2. Extracts subtitle texts and aligns them with frames.
  3. Uses heuristic rules to "parse" subject and verb components.
  4. Fills one of several manually written question templates.
  5. Saves the result in a JSON array format, streaming to avoid memory bloat.

---

#### ❌ Limitations and Cons

- **Low Question Quality (Garbage Output)**:
  - No semantic understanding of the subtitle content.
  - Template-based generation leads to repetitive and often meaningless or awkward phrasing.
  - Many generated questions are nonsensical, especially when subtitles are short, vague, or informal.
  - No actual language model (e.g., T5, GPT) is used for question generation.

- **Minimal Parsing Logic**:
  - Subject and verb extraction is based on crude regex rules.
  - Cannot distinguish between meaningful phrases and filler words.
  - Fails on edge cases like quotes, technical terms, or multiline subtitles.

- **Hardcoded Assumptions**:
  - Assumes subtitles are clean and structured.
  - No real NLP pipeline (no POS tagging, dependency parsing, etc.).
  - Limited support for variations in subtitle formats or structure.

- **Manual Template Fragility**:
  - Templates are rigid and often produce grammatically incorrect results if parsing fails.
  - Results are heavily biased by template phrasing and lack linguistic diversity.

- **Frame Storage as `.txt` Files**:
  - Saving base64 frames to `.txt` is inefficient and not scalable.
  - Not optimized for integration with image models (should store in image format or MongoDB).

---

#### 📦 Suggested Improvements

- Replace regex-based parsing with `spaCy` or `nltk` for subject-verb-object detection.
- Use pretrained models like T5, BART, or GPT for natural question generation.
- Store questions alongside compressed image data in MongoDB or an HDF5 format.
- Add quality filters to discard low-information subtitles.
- Consider batching subtitles for more context-aware questions.

---

> ⚠️ **Note:** This script is more of a placeholder for a question generation pipeline. It is useful for testing I/O and pipeline structure, but not suitable for real-world training without significant upgrades.


In [100]:
import os
import json
import re
import random
from pathlib import Path
import logging
from tqdm import tqdm

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class SubtitleQuestionGenerator:
    def __init__(self, frames_dir, output_file):
        """
        Initialize the question generator.
        
        Args:
            frames_dir: Directory containing frame JSON files from VideoProcessor
            output_file: Path to save the generated questions dataset
        """
        self.frames_dir = Path(frames_dir)
        self.output_file = output_file
        self.question_templates = [
            "What is happening when {subject} {verb}?",
            "Why does {subject} {verb}?",
            "Who is {verb_ing}?",
            "What does it mean when {subject} says \"{text}\"?",
            "Based on the subtitle \"{text}\", what is being discussed?",
            "What is the context of the statement \"{text}\"?",
            "How would you describe the scene where {subject} says \"{text}\"?",
            "What might happen after {subject} {verb}?",
            "What is the significance of \"{text}\"?",
            "What can you infer from the statement \"{text}\"?"
        ]
        
        # Create separate files for questions and frames
        self.base_dir = os.path.dirname(os.path.abspath(output_file))
        self.questions_file = output_file
        self.frames_dir_output = os.path.join(self.base_dir, "frame_images")
        os.makedirs(self.frames_dir_output, exist_ok=True)
        
        # Keep track of stats
        self.total_questions = 0
        self.total_frames = 0
        self.max_frame_size = 0
        
    def generate_questions_dataset(self):
        """Generate questions based on subtitle data from all frame files."""
        frame_files = list(self.frames_dir.glob("*_frames.json"))
        if not frame_files:
            logger.error(f"No frame files found in {self.frames_dir}")
            return
            
        logger.info(f"Found {len(frame_files)} frame files to process")
        
        # Open file for writing questions (stream instead of keeping in memory)
        with open(self.questions_file, "w") as questions_file:
            # Start the JSON array
            questions_file.write("[\n")
            
            first_question = True
            
            for file_path in tqdm(frame_files, desc="Processing frame files"):
                try:
                    # Process each file individually
                    for question in self._process_frame_file(file_path):
                        if not first_question:
                            questions_file.write(",\n")
                        else:
                            first_question = False
                            
                        # Write the question directly to file instead of storing in memory
                        questions_file.write(json.dumps(question, indent=2))
                        self.total_questions += 1
                        
                except Exception as e:
                    logger.error(f"Error processing file {file_path.name}: {str(e)}")
            
            # End the JSON array
            questions_file.write("\n]")
        
        logger.info(f"Generated {self.total_questions} questions and saved to {self.questions_file}")
        logger.info(f"Saved {self.total_frames} frame images to {self.frames_dir_output}")
        logger.info(f"Maximum frame size encountered: {self.max_frame_size / 1024 / 1024:.2f} MB")
        
    def _sanitize_filename(self, filename):
        """Create a safe filename by replacing path separators and other problematic characters."""
        # Replace slashes with underscores
        safe_name = filename.replace('/', '_').replace('\\', '_')
        # Replace other potentially problematic characters
        safe_name = re.sub(r'[<>:"|?*]', '_', safe_name)
        return safe_name
        
    def _process_frame_file(self, file_path):
        """Process a single frame file and yield questions one by one."""
        try:
            with open(file_path, "r") as f:
                frame_data = json.load(f)
        except json.JSONDecodeError:
            logger.error(f"Invalid JSON in file: {file_path}")
            return
        except Exception as e:
            logger.error(f"Error opening file {file_path}: {str(e)}")
            return
            
        for frame in frame_data:
            video_key = frame.get("video_key", "unknown")
            frame_idx = frame.get("frame_idx", 0)
            frame_base64 = frame.get("frame_base64", "")
            subtitles = frame.get("subtitles", [])
            
            if not subtitles or not frame_base64:
                continue
            
            # Create a safe filename
            safe_video_key = self._sanitize_filename(video_key)
            frame_filename = f"{safe_video_key}_{frame_idx}.txt"
            frame_path = os.path.join(self.frames_dir_output, frame_filename)
            
            try:
                # Track the size of the frame data
                frame_size = len(frame_base64)
                self.max_frame_size = max(self.max_frame_size, frame_size)
                
                # Write frame base64 to a separate file
                with open(frame_path, "w") as frame_file:
                    frame_file.write(frame_base64)
                self.total_frames += 1
            except Exception as e:
                logger.error(f"Error saving frame image: {str(e)}")
                continue
                
            for subtitle in subtitles:
                subtitle_text = subtitle.get("text", "").strip()
                if not subtitle_text or len(subtitle_text) < 5:
                    continue
                
                # Generate questions for this subtitle
                for question in self._generate_questions_for_subtitle(subtitle_text, video_key, frame_idx, frame_filename):
                    yield question
    
    def _generate_questions_for_subtitle(self, subtitle_text, video_key, frame_idx, frame_filename):
        """Generate multiple questions based on a single subtitle."""
        # Basic text cleaning
        clean_text = re.sub(r'\s+', ' ', subtitle_text).strip()
        
        # Skip very short texts or special characters only
        if len(clean_text) < 5 or not re.search(r'[a-zA-Z]', clean_text):
            return
            
        # Try to identify subjects and verbs for more targeted questions
        parsed = self._simple_parse(clean_text)
        
        # Generate 1-3 questions per subtitle
        num_questions = min(3, max(1, len(clean_text) // 20))
        
        for _ in range(num_questions):
            # Select a template - favor templates that work with the parsed data
            if parsed["subject"] and parsed["verb"] and random.random() > 0.4:
                template = random.choice([t for t in self.question_templates if "{subject}" in t and ("{verb}" in t or "{verb_ing}" in t)])
            else:
                template = random.choice([t for t in self.question_templates if "{text}" in t])
            
            # Fill in the template
            question = template.format(
                subject=parsed["subject"] or "the person",
                verb=parsed["verb"] or "speaks",
                verb_ing=parsed["verb_ing"] or "speaking",
                text=clean_text
            )
            
            yield {
                "question": question,
                "subtitle": clean_text,
                "video_key": video_key,
                "frame_idx": frame_idx,
                "frame_filename": frame_filename,  # Reference to frame file instead of embedding base64
                "start_time": parsed.get("start_time", ""),
                "end_time": parsed.get("end_time", "")
            }
    
    def _simple_parse(self, text):
        """Simple text parsing to extract subjects and verbs."""
        # Initialize with default values
        result = {
            "subject": "",
            "verb": "",
            "verb_ing": ""
        }
        
        # Very simple subject extraction - first noun/pronoun
        subject_match = re.search(r'\b(I|you|he|she|they|we|the [a-zA-Z]+|[A-Z][a-z]+ [A-Z][a-z]+|[A-Z][a-z]+)\b', text)
        if subject_match:
            result["subject"] = subject_match.group(1)
        
        # Simple verb extraction
        verb_matches = re.findall(r'\b([a-zA-Z]{2,})(s|ed|ing)?\b', text)
        for verb, suffix in verb_matches:
            if suffix == 'ing':
                result["verb_ing"] = verb + suffix
                result["verb"] = verb
                break
            elif suffix == 's' or suffix == 'ed' or suffix == '':
                result["verb"] = verb + suffix
                result["verb_ing"] = verb + 'ing'
                break
        
        return result

if __name__ == "__main__":
    # Example usage:
    processed_frames_dir = r"C:\Users\sahil\Downloads\AIVideos\processed_data"  # Directory with frame JSON files
    output_questions_file = r"C:\Users\sahil\Downloads\AIVideos\questions\subtitle_questions.json"  # Output file for questions
    
    generator = SubtitleQuestionGenerator(processed_frames_dir, output_questions_file)
    generator.generate_questions_dataset()

2025-04-04 13:00:02,483 - INFO - Found 8 frame files to process
Processing frame files: 100%|██████████| 8/8 [01:05<00:00,  8.13s/it]
2025-04-04 13:01:07,531 - INFO - Generated 5357210 questions and saved to C:\Users\sahil\Downloads\AIVideos\questions\subtitle_questions.json
2025-04-04 13:01:07,531 - INFO - Saved 1906 frame images to C:\Users\sahil\Downloads\AIVideos\questions\frame_images
2025-04-04 13:01:07,531 - INFO - Maximum frame size encountered: 0.45 MB


### 🤖 Enhanced Subtitle-Based Question Generation Using Gemma (Ollama)

This upgraded module replaces template-based question generation with **LLM-driven generation** using the locally hosted `Gemma3` model via **Ollama**. It is designed to extract rich, context-aware questions from subtitle-aligned frames and produce a dataset of open-ended questions grounded in visual and textual context.

---

#### ⚙️ Functionality Overview

- **Input**: JSON files with base64-encoded frames and aligned subtitles.
- **Output**:
  - A single `.json` file (`gemma_questions.json`) with generated questions, subtitles, and metadata.
  - Individual `.txt` files storing base64 frame data for each subtitle-question pair.

- **Workflow**:
  1. Parses each subtitle and sends it to Gemma via Ollama.
  2. Uses an instruction-style prompt to request multiple high-quality questions.
  3. Cleans and parses the model output, falling back to regex parsing or default questions if the response is malformed.
  4. Saves questions and relevant metadata (timestamps, frame references).

---

#### ✅ Improvements Over Baseline

- ✅ Uses LLM to generate **semantically richer** and more varied questions.
- ✅ Handles malformed or unexpected responses gracefully via fallback strategies.
- ✅ Designed to be local and offline-capable using Ollama + Gemma.
- ✅ Modular class structure with clear separation of responsibilities.
- ✅ Streamed JSON writing for memory efficiency on large datasets.

---

#### ❌ Limitations and Cons

- ⚠️ **Frame storage is still inefficient**:
  - Base64 frames are saved to `.txt` files individually, which is not scalable for larger datasets or multimodal training.
  - Ideal alternatives: store frames in MongoDB, convert to image files, or encode into LMDB/WebDataset.

- ⚠️ **Ollama latency and batch limitations**:
  - Gemma3 responses may be slow, especially when generating multiple questions.
  - No parallel request handling; questions are generated sequentially.

- ⚠️ **LLM hallucination and inconsistency**:
  - The model may sometimes ignore instructions and generate answers or commentary instead of pure questions.
  - JSON format enforcement is best-effort; Gemma responses vary in structure.

- ⚠️ **Subtitle context is limited**:
  - Only individual subtitle lines are processed.
  - Broader temporal or semantic context across multiple subtitles is not used.

- ⚠️ **No vision-language grounding yet**:
  - Although the frame is aligned and referenced, the LLM doesn't "see" the image — only text is used for generation.

---

#### 💡 Suggestions for Future Enhancement

- Integrate vision-language models (e.g., BLIP, Flamingo) for multimodal QA.
- Improve frame handling (store as compressed images or database BLOBs).
- Add spaCy/NLP parsing to filter low-quality subtitle inputs.
- Use sentence segmentation and context windows for multi-line subtitle merging.
- Batch Ollama queries (if model permits) for faster generation.

---

> This module demonstrates how locally hosted LLMs can be used to generate scalable QA datasets from video subtitles — a powerful step toward multimodal AI assistants.


In [21]:
import os
import json
import re
from pathlib import Path
import logging
from tqdm import tqdm
import ollama

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class EnhancedSubtitleQuestionGenerator:
    def __init__(self, frames_dir, output_file, model="gemma3:latest"):
        """
        Initialize the enhanced question generator.
        
        Args:
            frames_dir: Directory containing frame JSON files from VideoProcessor
            output_file: Path to save the generated questions dataset
            model: Model name to use (Gemma3 model via Ollama)
        """
        self.frames_dir = Path(frames_dir)
        self.output_file = output_file
        self.model = model
        
        # Create output directories
        self.base_dir = os.path.dirname(os.path.abspath(output_file))
        self.questions_file = output_file
        self.frames_dir_output = os.path.join(self.base_dir, "frame_images")
        os.makedirs(self.frames_dir_output, exist_ok=True)
        
        # Track stats
        self.total_questions = 0
        self.total_frames = 0
        self.max_frame_size = 0
        
        # Improved system prompt for Gemma3
        self.question_prompt = """[INST]
        Generate {num_questions} questions based on this video subtitle:
        "{subtitle_text}"
        
        Requirements:
        - Focus on visual context and implications
        - Ask open-ended questions
        - Vary question types
        - Return ONLY a JSON array of questions
        
        Example: ["Question 1?", "Question 2?", "Question 3?"]
        [/INST]"""

    def generate_questions_dataset(self):
        """Generate questions based on subtitle data from all frame files."""
        frame_files = list(self.frames_dir.glob("*_frames.json"))
        if not frame_files:
            logger.error(f"No frame files found in {self.frames_dir}")
            return
            
        logger.info(f"Found {len(frame_files)} frame files to process")
        
        with open(self.questions_file, "w") as questions_file:
            questions_file.write("[\n")
            first_question = True
            
            for file_path in tqdm(frame_files, desc="Processing frame files"):
                try:
                    with open(file_path, "r") as f:
                        frame_data = json.load(f)
                    
                    for question in self._process_frames(frame_data):
                        if not first_question:
                            questions_file.write(",\n")
                        else:
                            first_question = False
                        
                        questions_file.write(json.dumps(question, indent=2))
                        self.total_questions += 1
                        
                except Exception as e:
                    logger.error(f"Error processing {file_path.name}: {str(e)}")
            
            questions_file.write("\n]")
        
        logger.info(f"Generated {self.total_questions} questions")
        logger.info(f"Saved {self.total_frames} frame images")

    def _process_frames(self, frame_data):
        """Process frames and generate questions."""
        for frame in frame_data:
            if not frame.get("subtitles") or not frame.get("frame_base64"):
                continue
            
            # Save frame data
            frame_filename = self._save_frame_data(frame)
            if not frame_filename:
                continue
            
            # Process subtitles
            for subtitle in frame["subtitles"]:
                clean_text = self._clean_subtitle(subtitle.get("text", ""))
                if not clean_text:
                    continue
                
                yield from self._generate_questions(
                    clean_text,
                    frame["video_key"],
                    frame["frame_idx"],
                    frame_filename,
                    subtitle
                )

    def _save_frame_data(self, frame):
        """Save frame image data and return filename."""
        try:
            safe_name = re.sub(r'[<>:"/\\|?*]', '_', frame["video_key"])
            frame_filename = f"{safe_name}_{frame['frame_idx']}.txt"
            frame_path = os.path.join(self.frames_dir_output, frame_filename)
            
            with open(frame_path, "w") as f:
                f.write(frame["frame_base64"])
            
            self.total_frames += 1
            self.max_frame_size = max(
                self.max_frame_size, 
                len(frame["frame_base64"])
            )
            return frame_filename
        except Exception as e:
            logger.error(f"Error saving frame: {str(e)}")
            return None

    def _generate_questions(self, subtitle_text, video_key, frame_idx, frame_filename, subtitle):
        """Generate questions using Gemma3 via Ollama."""
        num_questions = min(3, max(1, len(subtitle_text) // 20))
        
        try:
            response = ollama.generate(
                model=self.model,
                prompt=self.question_prompt.format(
                    subtitle_text=subtitle_text,
                    num_questions=num_questions
                ),
                options={
                    "temperature": 0.7,
                    "max_tokens": 500
                }
            )
            
            questions = self._parse_response(response['response'])
            
            for q in questions:
                yield {
                    "question": q,
                    "subtitle": subtitle_text,
                    "video_key": video_key,
                    "frame_idx": frame_idx,
                    "frame_filename": frame_filename,
                    **{k: subtitle.get(k, "") for k in ["start_time", "end_time"]}
                }
                
        except Exception as e:
            logger.error(f"LLM Error: {str(e)}")
            yield self._fallback_question(subtitle_text, video_key, frame_idx, frame_filename, subtitle)

    def _parse_response(self, response_text):
        """Parse LLM response into questions."""
        try:
            # First try direct JSON parsing
            return json.loads(response_text)
        except json.JSONDecodeError:
            # Try extracting JSON array
            match = re.search(r'\[.*\]', response_text, re.DOTALL)
            if match:
                try:
                    return json.loads(match.group(0))
                except json.JSONDecodeError:
                    pass
                    
            # Fallback to line-based extraction
            return [
                q.strip() for q in response_text.split('\n') 
                if q.strip().endswith('?') and len(q.strip()) > 10
            ]

    def _clean_subtitle(self, text):
        """Clean and validate subtitle text."""
        clean_text = re.sub(r'\s+', ' ', text).strip()
        return clean_text if len(clean_text) >= 5 and re.search(r'[a-zA-Z]', clean_text) else None

    def _fallback_question(self, text, video_key, frame_idx, filename, subtitle):
        """Generate fallback question on error."""
        return {
            "question": f"What context might surround the subtitle: '{text}'?",
            "subtitle": text,
            "video_key": video_key,
            "frame_idx": frame_idx,
            "frame_filename": filename,
            **{k: subtitle.get(k, "") for k in ["start_time", "end_time"]}
        }

if __name__ == "__main__":
    generator = EnhancedSubtitleQuestionGenerator(
        r"C:\Users\sahil\Downloads\AIVideos\processed_data",
        r"C:\Users\sahil\Downloads\AIVideos\questions\gemma_questions.json"
    )
    generator.generate_questions_dataset()

2025-04-04 13:54:10,283 - INFO - Found 8 frame files to process
Processing frame files:   0%|          | 0/8 [00:00<?, ?it/s]2025-04-04 13:54:11,664 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:12,196 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:13,382 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:13,998 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:15,093 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:15,681 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:16,730 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:17,219 - INFO - HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
2025-04-04 13:54:18,393 - I

KeyboardInterrupt: 

In [9]:
import requests

def test_ollama_gemma3():
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:latest",  # Use the exact model name you pulled
        "prompt": "Explain quantum computing in one sentence.",
        "stream": False  # Set to True if you want streaming response
    }

    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        print("Success! Response from Gemma3:")
        print(response.json()["response"])
        
    except requests.exceptions.ConnectionError:
        print("Connection failed. Is Ollama running?")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error: {e}")
    except KeyError:
        print("Unexpected response format")

if __name__ == "__main__":
    test_ollama_gemma3()

Success! Response from Gemma3:
Quantum computing leverages the principles of quantum mechanics, like superposition and entanglement, to perform complex calculations far beyond the capabilities of classical computers by manipulating qubits that can exist in multiple states simultaneously.


### 🧠 Efficient Subtitle-to-Question Generation with Ollama (Gemma3) and MongoDB

This script introduces a **multi-threaded subtitle processing pipeline** that leverages the **Gemma3 model hosted locally via Ollama** to convert subtitle segments into natural language questions. Unlike previous methods, it uses MongoDB to source subtitle data and store questions, ensuring **modularity**, **scalability**, and **fault tolerance**.

---

#### 🚀 Key Features

- **MongoDB Integration**:
  - Reads subtitle segments directly from the `subtitles` collection (`video_key`, `subtitles`).
  - Stores generated questions in the `questions` collection with timestamps and source context.
  - Supports upsert, avoiding duplicates and enabling incremental updates.

- **Gemma3 LLM via Ollama**:
  - Prompt design encourages 3–5 **natural-sounding, relevant questions** per subtitle.
  - Exponential backoff and retry logic ensures robustness against LLM failures.

- **Text Preprocessing**:
  - Cleans filler words (e.g., “um”, “uh”, “like”) for better input clarity.
  - Rejects subtitles that are too short to provide meaningful context.

- **Multithreading with ThreadPoolExecutor**:
  - Efficiently parallelizes subtitle processing to reduce wall-clock time.
  - Uses `max_workers` to adapt to local hardware constraints.

- **Postprocessing and Extraction**:
  - Filters LLM output to retain only well-formed, question-marked lines with sufficient length.

---

#### ✅ Why This Is the Best Approach (So Far)

| Feature | Benefit |
|--------|---------|
| 🧩 **MongoDB-driven** | Enables persistent, indexed storage and integration with the rest of the data pipeline. |
| ⚙️ **LLM-based generation (via Ollama)** | Produces context-aware, human-like questions, customizable via prompt. |
| 🧼 **Precleaned subtitle inputs** | Avoids garbage in → garbage out issues common with filler-laden text. |
| 🔁 **Retry and resilience mechanisms** | Avoids crashes or failures due to transient Ollama issues. |
| 🧵 **Multithreading** | Significantly speeds up question generation for long subtitle files. |
| 🛠 **Minimal dependencies, runs locally** | Doesn't rely on external APIs or cloud services—fully offline-capable. |

---

#### ⚠️ Potential Areas for Improvement

- Still **LLM-only**, no image context used (though aligned via `video_key` and `timestamps`).
- Quality depends on **Gemma3** model capabilities—may improve with fine-tuning or model switching.
- Could benefit from **contextual windowing** (e.g., 2–3 subtitle segments for better questions).
- **Hardcoded prompt** might limit variability across different types of subtitle content.

---

> In summary, this pipeline represents a **robust, scalable, and well-structured** approach to building high-quality question-answer datasets from raw subtitles. Its integration with a local database and concurrency support makes it suitable for large-scale processing and deployment-ready AI systems.


In [1]:
import json
import requests
import re
from time import sleep
from pymongo import MongoClient
from tqdm import tqdm
import concurrent.futures

# Configuration
OLLAMA_MODEL = "gemma3"
OLLAMA_URL = "http://localhost:11434/api/generate"

# MongoDB setup
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
sub_collection = db["subtitles"]
qa_collection = db["questions"]

def clean_text(text):
    """Remove filler words, repeated spaces, and noise from subtitle text."""
    text = re.sub(r'\b(uh+|um+|like|okay|so|right|you know)\b', '', text, flags=re.IGNORECASE)
    return re.sub(r'\s+', ' ', text).strip()

def extract_questions(text):
    """Extract all valid question lines from LLM output."""
    questions = []
    for line in text.splitlines():
        line = line.strip(" *-–\t")
        if "?" in line and len(line.split()) >= 4:
            questions.append(line)
    return questions

def prompt_gemma(text, retries=2):
    """Send cleaned subtitle text to Ollama and extract questions."""
    payload = {
        "model": OLLAMA_MODEL,
        "prompt": f"Generate 3 to 5 natural-sounding questions based on this subtitle:\n\"{text}\"",
        "stream": False
    }
    for attempt in range(retries + 1):
        try:
            res = requests.post(OLLAMA_URL, json=payload, timeout=60)
            res.raise_for_status()
            return extract_questions(res.json()["response"].strip())
        except Exception as e:
            print(f"⚠️ Retry {attempt} failed for: {text[:60]}... → {e}")
            sleep(2 ** attempt)  # Exponential backoff
    return []

def process_subtitle(subtitle):
    """Process a single subtitle segment and return its question set."""
    if not subtitle.get("text"):
        return None

    raw = subtitle["text"].strip().replace('\n', ' ')
    text = clean_text(raw)
    if len(text.split()) < 4:
        return None

    questions = prompt_gemma(text)
    if questions:
        return {
            "questions": questions,
            "source": text,
            "timestamp": {
                "start": subtitle["start"],
                "end": subtitle["end"]
            }
        }

def generate_questions_for_video(video_doc, max_workers=6):
    """Generate and store all questions for a video using concurrent subtitle processing."""
    video_key = video_doc["video_key"]
    subtitles = video_doc["subtitles"]
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_subtitle, s) for s in subtitles]
        for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=f"[{video_key}]"):
            result = f.result()
            if result:
                results.append(result)

    if results:
        qa_collection.update_one(
            {"video_key": video_key},
            {"$set": {"questions": results}},
            upsert=True
        )
        print(f"✅ Stored {len(results)} questions for {video_key}")
    else:
        print(f"⚠️ No valid questions generated for {video_key}")

# Optional runner (if needed in a main loop)
if __name__ == "__main__":
    for video_doc in sub_collection.find():
        generate_questions_for_video(video_doc, max_workers=6)



[9CGGh6ivg68]: 100%|██████████| 204/204 [05:44<00:00,  1.69s/it]


✅ Stored 201 questions for 9CGGh6ivg68


[WXoOohWU28Y]: 100%|██████████| 582/582 [15:29<00:00,  1.60s/it]


✅ Stored 568 questions for WXoOohWU28Y


[rCVlIVKqqGE]: 100%|██████████| 592/592 [15:34<00:00,  1.58s/it]


...

✅ Stored 341 questions for lb_5AdUpfuA


[eFgkZKhNUdM]: 100%|██████████| 886/886 [21:02<00:00,  1.42s/it]


✅ Stored 800 questions for eFgkZKhNUdM


[eQ6UE968Xe4]: 100%|██████████| 1892/1892 [47:18<00:00,  1.50s/it]

✅ Stored 1796 questions for eQ6UE968Xe4


### 🕒 Adding Human-Readable Timestamps to Video Frames in MongoDB

This script enriches each stored video frame in the `video_frames` MongoDB collection with a **human-readable timestamp**, calculated from the frame index and a default frame rate (FPS). This is particularly useful for aligning frame data with subtitles, video playback, or temporal analysis.

---

#### ⚙️ How It Works

1. **FPS-Based Time Calculation**:
   - A default FPS (frames per second) value is used to convert frame indices to time in seconds.
   - The `seconds_to_timestamp` function then formats the time into `HH:MM:SS.mmm`.

2. **MongoDB Update Logic**:
   - Iterates over all distinct `video_key` entries in the database.
   - For each video, retrieves its frames and updates them with a new `timestamp` field.

3. **Timestamp Format**:
   - Stored as a string like `"00:01:23.456"` for easy reading and integration with subtitle or clip slicing workflows.

---

#### 🧩 Example Output

For a frame with index `1234` at 30 FPS:
- Time in seconds: `1234 / 30 = 41.13`
- Timestamp stored: `"00:00:41.130"`

---

#### ✅ Benefits

- Enables **precise temporal alignment** between visual frames and subtitles.
- Useful for **search, visualization, and retrieval systems** that show clips based on time.
- Adds **semantic context** to each frame without requiring external tools.

---

> This simple utility greatly enhances the structure and accessibility of your frame-level dataset by embedding time context directly into the database.


In [1]:
from pymongo import MongoClient
from datetime import timedelta

# Assume same FPS for all for now
DEFAULT_FPS = 30  # Change if needed

def seconds_to_timestamp(seconds):
    td = timedelta(seconds=seconds)
    hours, remainder = divmod(td.seconds, 3600)
    minutes, secs = divmod(remainder, 60)
    millis = int(td.microseconds / 1000)
    return f"{hours:02}:{minutes:02}:{secs:02}.{millis:03}"

client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
frames_col = db["video_frames"]

# Step 1: Get all unique video_keys
video_keys = frames_col.distinct("video_key")

# Step 2: Update each video’s frames
for video_key in video_keys:
    fps = DEFAULT_FPS  # You can customize this per video if needed

    frames = frames_col.find({"video_key": video_key})
    for frame in frames:
        idx = frame["frame_idx"]
        ts_str = seconds_to_timestamp(idx / fps)

        frames_col.update_one(
            {"_id": frame["_id"]},
            {"$set": {"timestamp": ts_str}}
        )
    print(f"✅ Added timestamps for {video_key}")


✅ Added timestamps for videos_9CGGh6ivg68_9CGGh6ivg68
✅ Added timestamps for videos_FCQ-rih6cHY_FCQ-rih6cHY
✅ Added timestamps for videos_TV-DjM8242s_TV-DjM8242s
✅ Added timestamps for videos_WXoOohWU28Y_WXoOohWU28Y
✅ Added timestamps for videos_eFgkZKhNUdM_eFgkZKhNUdM
✅ Added timestamps for videos_eQ6UE968Xe4_eQ6UE968Xe4
✅ Added timestamps for videos_lb_5AdUpfuA_lb_5AdUpfuA
✅ Added timestamps for videos_rCVlIVKqqGE_rCVlIVKqqGE


### 🎞️ Frame Extraction at Question Timestamps

This script extracts **a single representative frame** for each question in the MongoDB `questions` collection by computing the **midpoint** between subtitle start and end times. It uses `ffmpeg` to extract these keyframes and updates MongoDB with both the frame path and its precise timestamp.

---

#### 💡 Purpose

To **visually anchor** each generated question with a corresponding video frame, making it easier to:

- Display a visual preview for each question.
- Link semantic understanding (subtitles/questions) with visual cues.
- Prepare data for fine-tuning multimodal models.

---

#### ⚙️ Features

- **Precise timestamping** using subtitle boundaries and `datetime` midpoint calculation.
- **Batch processing** of all `.mp4` files under a dataset directory.
- **MongoDB integration** to append `frame_path` and `frame_timestamp` to each question document.
- **FFmpeg-based extraction** for efficient and high-quality frame grabbing.

---

#### 📁 Output

Each video creates a directory under the specified output path:
```

frames\_output/
├── lecture1/
│   ├── q0000.jpg
│   ├── q0001.jpg
│   └── ...
├── lecture2/
│   └── q0000.jpg

```

Each image corresponds to the midpoint of a subtitle segment associated with a question.

---

#### ✅ Benefits

- Adds **rich visual context** to questions for UI display or training.
- Makes the question-answer pipeline **more interpretable and explainable**.
- Keeps **MongoDB up-to-date** with consistent frame-level metadata.

---

> This is a key step in preparing a multimodal dataset where language and vision align at fine-grained points in a lecture video.

In [1]:
import os
import subprocess
import logging
from pymongo import MongoClient
from datetime import datetime
from tqdm import tqdm

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# MongoDB setup
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
questions_col = db["questions"]

def get_midpoint_timestamp(start, end):
    fmt = "%H:%M:%S.%f"
    t1 = datetime.strptime(start, fmt)
    t2 = datetime.strptime(end, fmt)
    mid = t1 + (t2 - t1) / 2
    return mid.strftime("%H:%M:%S.%f")[:-3]

def extract_frame(video_path, timestamp, output_path):
    cmd = [
        "ffmpeg", "-ss", timestamp,
        "-i", video_path,
        "-frames:v", "1",
        "-q:v", "2",
        "-y",
        output_path
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

def find_all_video_files(dataset_dir):
    """Find all .mp4 files under the dataset directory."""
    video_paths = {}
    for root, _, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith(".mp4"):
                video_key = os.path.splitext(file)[0]
                full_path = os.path.join(root, file)
                video_paths[video_key] = full_path
                logger.info(f"Found video: {video_key} at {full_path}")
    return video_paths

def extract_frames_for_questions(video_key, video_path, frame_output_dir):
    video_doc = questions_col.find_one({"video_key": video_key})
    if not video_doc:
        logger.warning(f"No MongoDB entry found for video_key: {video_key}")
        return

    frame_dir = os.path.join(frame_output_dir, video_key)
    os.makedirs(frame_dir, exist_ok=True)
    logger.info(f"Extracting frames for {video_key} into {frame_dir}")

    updated_questions = []
    for i, q in enumerate(video_doc["questions"]):
        ts_start = q["timestamp"]["start"]
        ts_end = q["timestamp"]["end"]
        ts_mid = get_midpoint_timestamp(ts_start, ts_end)

        frame_name = f"q{i:04d}.jpg"
        frame_path = os.path.join(frame_dir, frame_name)
        relative_path = os.path.relpath(frame_path, os.getcwd())

        extract_frame(video_path, ts_mid, frame_path)

        q["frame_path"] = relative_path
        q["frame_timestamp"] = ts_mid
        updated_questions.append(q)

        logger.info(f"Extracted frame {frame_name} at {ts_mid}")

    # Update MongoDB
    questions_col.update_one(
        {"_id": video_doc["_id"]},
        {"$set": {"questions": updated_questions}}
    )
    logger.info(f"✅ Frame extraction complete for {video_key} ({len(updated_questions)} frames)")

def extract_all_frames_from_dataset(dataset_dir, frame_output_dir):
    logger.info(f"Scanning for video files in: {dataset_dir}")
    video_paths = find_all_video_files(dataset_dir)
    logger.info(f"Found {len(video_paths)} video(s)")

    for video_key, video_path in tqdm(video_paths.items(), desc="Processing videos"):
        extract_frames_for_questions(video_key, video_path, frame_output_dir)

if __name__ == "__main__":
    dataset_dir = r"C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data"  # Same as subtitle input
    frame_output_dir = r"C:\\Users\\sahil\\Downloads\\AIVideos\\frames_output"
    extract_all_frames_from_dataset(dataset_dir, frame_output_dir)
    logger.info("Frame extraction and MongoDB update completed successfully.")


INFO:__main__:Scanning for video files in: C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data
INFO:__main__:Found video: 9CGGh6ivg68 at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000000\videos\9CGGh6ivg68\9CGGh6ivg68.mp4
INFO:__main__:Found video: WXoOohWU28Y at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000000\videos\WXoOohWU28Y\WXoOohWU28Y.mp4
INFO:__main__:Found video: rCVlIVKqqGE at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000001\videos\rCVlIVKqqGE\rCVlIVKqqGE.mp4
INFO:__main__:Found video: TV-DjM8242s at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000001\videos\TV-DjM8242s\TV-DjM8242s.mp4
INFO:__main__:Found video: FCQ-rih6cHY at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000002\videos\FCQ-rih6cHY\FCQ-rih6cHY.mp4
INFO:__main__:Found video: lb_5AdUpfuA at C:\\Users\\sahil\\Downloads\\AIVideos\\processed_data\shard-000002\videos\lb_5AdUpfuA\lb_5AdUpfuA.mp4
INFO:__main__:Found video: eFgkZKhNUdM 

## Sentence Embedding

#### Step 1: Merging Subtitle Segments into Sentences

Raw subtitle data often consists of short, fragmented lines not well-suited for semantic analysis or question generation. In this step, we use `spaCy` to intelligently segment the subtitles into meaningful, full-length sentences. We align these sentences with the original subtitle timestamps, preserving temporal context for downstream tasks like frame extraction or video indexing.

This preprocessing step is stored in a MongoDB collection (`merged_sentences`) for reusability.


In [1]:
import spacy
from pymongo import MongoClient
from tqdm import tqdm

nlp = spacy.load("en_core_web_trf")

client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
subs_col = db["subtitles"]
merged_col = db["merged_sentences"]

def merge_subtitles(subtitles):
    full_text = " ".join([s["text"].replace("\n", " ") for s in subtitles])
    doc = nlp(full_text)
    return [sent.text.strip() for sent in doc.sents]

def align_timestamps(sentences, subtitles):
    aligned = []
    sub_idx = 0
    sub_texts = [s["text"].replace("\n", " ").strip() for s in subtitles]

    for sent in sentences:
        matched_subs = []
        while sub_idx < len(sub_texts) and len(" ".join(matched_subs)) < len(sent):
            matched_subs.append(sub_texts[sub_idx])
            sub_idx += 1

        if matched_subs:
            start = subtitles[sub_idx - len(matched_subs)]["start"]
            end = subtitles[sub_idx - 1]["end"]
            aligned.append({
                "sentence": sent,
                "start": start,
                "end": end
            })
    return aligned

def process_video(doc):
    video_key = doc["video_key"]
    subtitles = doc["subtitles"]

    spacy_sentences = merge_subtitles(subtitles)
    timestamped = align_timestamps(spacy_sentences, subtitles)

    merged_col.update_one(
        {"video_key": video_key},
        {"$set": {"sentences": timestamped}},
        upsert=True
    )
    print(f"✅ {video_key}: {len(timestamped)} merged sentences")

if __name__ == "__main__":
    for doc in tqdm(subs_col.find(), desc="Merging subtitles"):
        process_video(doc)


c:\Users\sahil\anaconda3\Lib\site-packages\thinc\shims\pytorch.py:261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filelike, map_location=

✅ 9CGGh6ivg68: 6 merged sentences


Merging subtitles: 2it [00:03,  1.83s/it]

✅ WXoOohWU28Y: 13 merged sentences


Merging subtitles: 3it [00:05,  2.13s/it]

...

✅ lb_5AdUpfuA: 16 merged sentences


Merging subtitles: 7it [00:21,  3.19s/it]

✅ eFgkZKhNUdM: 15 merged sentences


Merging subtitles: 8it [00:29,  3.71s/it]

✅ eQ6UE968Xe4: 28 merged sentences


#### Step 2: Running Topic Modeling on Merged Sentences

With properly segmented and timestamped sentences from Step 1, we now apply topic modeling using `BERTopic`. This helps identify latent themes or recurring topics across the video. Each sentence is cleaned, embedded, and assigned a topic using a transformer-based encoder.

This step allows us to group related content and improve downstream retrieval or question-answering quality. The results here are only printed and not yet stored.


In [ ]:
from pymongo import MongoClient
from bertopic import BERTopic
import re

# Step 1: Clean sentence function
def clean_sentence(text):
    # Remove filler words and extra spaces
    text = re.sub(r"\b(uh+|um+|you know|like|okay|right|so|hmm|just)\b", "", text, flags=re.IGNORECASE)
    return re.sub(r'\s+', ' ', text).strip()

# Step 2: Connect to MongoDB and fetch merged subtitles
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
merged_col = db["merged_sentences"]

# Choose a video that has many merged sentences
video_key = "TV-DjM8242s"  # Replace with any other if needed
doc = merged_col.find_one({"video_key": video_key})
raw_sentences = [s["sentence"] for s in doc["sentences"]]

# Clean the sentences
sentences = [clean_sentence(s) for s in raw_sentences if len(s.strip()) > 0]

# Step 3: Run BERTopic
topic_model = BERTopic()
topics, _ = topic_model.fit_transform(sentences)

# Step 4: Show results
print(f"\n📊 Topics for video: {video_key}")
for sent, topic in zip(sentences, topics):
    print(f"[Topic {topic}] {sent}")



📊 Topics for video: TV-DjM8242s
[Topic 0] this is basically what is happening this is basically what is happening in a an example over here where we in a an example over here where we in a an example over here where we have an input have an input have an input image and we are actually sliding image and we are actually sliding image and we are actually sliding a kernel a 3X3 kernel and we are a kernel a 3X3 kernel and we are a kernel a 3X3 kernel and we are getting an an an output feature map getting an an an output feature map getting an an an output feature map
[Topic 1] 
[Topic 0] the input feature map here is the input feature map here is the input feature map here is has one we will be calling sometimes has one we will be calling sometimes has one we will be calling sometimes this depth Channel this depth Channel this depth Channel and the output feature map has again and the output feature map has again and the output feature map has again one channel over one channel over one c

#### Step 3: Merging, Segmenting, and Topic Modeling in One Pipeline

To streamline the pipeline, this final step consolidates merging, segmentation, and topic modeling into a single, optimized script. It uses a `SentenceTransformer` for high-quality embeddings and `UMAP` for dimensionality reduction. Sentences are directly mapped to topics and aligned with timestamps, and the results are stored in a new MongoDB collection (`topics`).

This all-in-one script eliminates the need for intermediate collections and enables scalable topic-aware video search.


In [6]:
import os
import spacy
from pymongo import MongoClient
from bertopic import BERTopic
from umap import UMAP
from sentence_transformers import SentenceTransformer

# Load spaCy model
nlp = spacy.load("en_core_web_trf")

# MongoDB connection
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
subs_col = db["subtitles"]
topic_col = db["topics"]

# Create embedding and topic model
embedding_model = SentenceTransformer("all-mpnet-base-v2")
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=UMAP(n_neighbors=3, n_components=2, min_dist=0.0, metric="cosine"),
    calculate_probabilities=False,
    verbose=True
)

def merge_subtitles(subtitles):
    """Merge short subtitle segments and clean text."""
    merged = []
    buffer = ""
    start_time = None
    for sub in subtitles:
        text = sub["text"].strip().replace("\n", " ")
        if not text:
            continue
        if start_time is None:
            start_time = sub["start"]
        buffer += " " + text
        if len(buffer.split()) > 20:
            merged.append({"text": buffer.strip(), "start": start_time, "end": sub["end"]})
            buffer = ""
            start_time = None
    if buffer:
        merged.append({"text": buffer.strip(), "start": start_time, "end": subtitles[-1]["end"]})
    return merged

def segment_sentences(text):
    """Segment a block of text into clean sentences."""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if len(sent.text.strip().split()) > 4]

def process_video(doc):
    video_key = doc["video_key"]
    subtitles = doc["subtitles"]
    merged = merge_subtitles(subtitles)
    
    timestamped = []
    for item in merged:
        for sent in segment_sentences(item["text"]):
            timestamped.append({
                "text": sent,
                "video_key": video_key,
                "start": item["start"],
                "end": item["end"]
            })

    if not timestamped:
        print(f"⚠️ Skipping {video_key}, no sentences found.")
        return

    sentences = [item["text"] for item in timestamped]
    topics, _ = topic_model.fit_transform(sentences)
    for i, t in enumerate(topics):
        timestamped[i]["topic"] = int(t)

    topic_col.update_one(
        {"video_key": video_key},
        {"$set": {"sentences": timestamped}},
        upsert=True
    )

    print(f"✅ {video_key}: {len(timestamped)} merged sentences")

if __name__ == "__main__":
    for doc in subs_col.find():
        process_video(doc)

2025-05-06 02:49:57,759 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2025-05-06 02:49:58,656 - BERTopic - Embedding - Completed ✓
2025-05-06 02:49:58,656 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-06 02:49:58,701 - BERTopic - Dimensionality - Completed ✓
2025-05-06 02:49:58,702 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-06 02:49:58,705 - BERTopic - Cluster - Completed ✓
2025-05-06 02:49:58,708 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-06 02:49:58,713 - BERTopic - Representation - Completed ✓


✅ 9CGGh6ivg68: 92 merged sentences


2025-05-06 02:50:04,999 - BERTopic - Embedding - Transforming documents to embeddings.


...

✅ eFgkZKhNUdM: 338 merged sentences


2025-05-06 02:51:34,590 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

2025-05-06 02:51:41,438 - BERTopic - Embedding - Completed ✓
2025-05-06 02:51:41,439 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-06 02:51:42,318 - BERTopic - Dimensionality - Completed ✓
2025-05-06 02:51:42,318 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-06 02:51:42,329 - BERTopic - Cluster - Completed ✓
2025-05-06 02:51:42,331 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-06 02:51:42,352 - BERTopic - Representation - Completed ✓


✅ eQ6UE968Xe4: 785 merged sentences


#### Step 4: Semantic Indexing with Sentence Embeddings in Qdrant

To enable fast and semantically meaningful search over video content, we embed all previously merged and timestamped sentences using the `all-MiniLM-L6-v2` model from SentenceTransformers. Each sentence is encoded into a 384-dimensional vector representation, capturing its contextual meaning.

These embeddings are stored in a Qdrant vector database, a high-performance similarity search engine. We recreate a Qdrant collection (`video_embeddings`) with cosine distance as the similarity metric. Each stored vector is accompanied by metadata including the `video_key`, `start` and `end` timestamps, and raw text content.

This step lays the foundation for real-time semantic search and video navigation, enabling downstream applications like question answering, contextual retrieval, and timeline-based recommendations.


In [13]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, PointStruct
from tqdm import tqdm
import uuid

# MongoDB setup
client = MongoClient("mongodb://localhost:27017/")
db = client["video_rag"]
merged_col = db["merged_sentences"]

# Qdrant setup
qdrant = QdrantClient(host="localhost", port=6333)

qdrant.recreate_collection(
    collection_name="video_embeddings",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),  # 384 for MiniLM
)

# Embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Prepare payloads and vectors
payloads, vectors = [], []
for doc in tqdm(merged_col.find(), desc="📄 Processing merged_sentences"):
    video_key = doc["video_key"]
    for sent in doc["sentences"]:
        text = sent.get("sentence", "").strip()  # ✅ Fix here
        if len(text.split()) < 3:
            continue
        embedding = model.encode(text)
        payloads.append({
            "video_key": video_key,
            "start": sent["start"],
            "end": sent["end"],
            "text": text
        })
        vectors.append(embedding)

# Upload to Qdrant
if vectors:
    points = [
        PointStruct(id=uuid.uuid4().hex, vector=vec, payload=payload)
        for vec, payload in zip(vectors, payloads)
    ]
    qdrant.upsert(collection_name="video_embeddings", points=points)
    print(f"✅ Uploaded {len(points)} sentence embeddings to Qdrant.")
else:
    print("⚠️ No valid sentences found to embed.")


C:\Users\sahil\AppData\Local\Temp\ipykernel_34372\1551695893.py:16: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
📄 Processing merged_sentences: 8it [00:02,  2.70it/s]


✅ Uploaded 130 sentence embeddings to Qdrant.


## Similarity Search

This script implements a minimal semantic search interface using a SentenceTransformer (`all-MiniLM-L6-v2`) and a Qdrant vector database.

- **Purpose**: Retrieve the top-K video transcript snippets semantically closest to a given question.
- **Process**:
  1. Encodes the input query into a vector.
  2. Searches the `video_embeddings` collection in Qdrant.
  3. Displays matched transcript snippets, along with associated metadata (video key, timestamps).
- **Usage**: A foundational test script for confirming embedding quality and database integrity before integrating any frontend or multimedia features.

> ✅ This lays the groundwork for retrieval-augmented applications like AI-based video assistants.


In [14]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

# Load model and connect to Qdrant
model = SentenceTransformer("all-MiniLM-L6-v2")
qdrant = QdrantClient(host="localhost", port=6333)

def search_similar_sentences(query, top_k=5):
    embedding = model.encode(query).tolist()

    results = qdrant.search(
        collection_name="video_embeddings",
        query_vector=embedding,
        limit=top_k
    )

    for i, res in enumerate(results):
        payload = res.payload
        print(f"\n🔍 Match {i+1}")
        print(f"Text: {payload['text']}")
        print(f"Video: {payload['video_key']}")
        print(f"Start: {payload['start']}, End: {payload['end']}")

    return results

# Example usage
query = "how does padding work in convolution?"
search_similar_sentences(query)



🔍 Match 1
Text: the first is the form uh of the the the first is the form uh of the the concept of padding and uh typically we concept of padding and uh typically we concept of padding and uh typically we are padding uh the U input uh feature are padding uh
Video: TV-DjM8242s
Start: 00:00:55.910, End: 00:01:05.040

🔍 Match 2
Text: not really make a lot of sense to you the moment you see this kind of the moment you see this kind of the moment you see this kind of Animation over here why in Earth we're Animation over here why in Earth we're Animation over here why in Earth we're going to do one by one convolutions uh going to do one by one convolutions uh going to do one by one convolutions uh since we as we discussed we're trying to since we as we discussed we're trying to since we as we discussed we're trying to detect features and typically the kernel detect features and typically the kernel detect features and typically the kernel sizes have larger Dimensions than one by sizes have 

[ScoredPoint(id='f5855e0b-b5e2-4ed6-ab78-5a3062deaf2d', version=0, score=0.4169496, payload={'video_key': 'TV-DjM8242s', 'start': '00:00:55.910', 'end': '00:01:05.040', 'text': 'the first is the form uh of the the the first is the form uh of the the concept of padding and uh typically we concept of padding and uh typically we concept of padding and uh typically we are padding uh the U input uh feature are padding uh'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id='9ac9117b-efdc-4754-a650-12dfeb3872ea', version=0, score=0.37636524, payload={'video_key': 'TV-DjM8242s', 'start': '00:31:41.350', 'end': '00:32:48.950', 'text': "not really make a lot of sense to you the moment you see this kind of the moment you see this kind of the moment you see this kind of Animation over here why in Earth we're Animation over here why in Earth we're Animation over here why in Earth we're going to do one by one convolutions uh going to do one by one convolutions uh going to do one by on

This version evolves the system into an interactive **Gradio app** that not only retrieves matching transcript text, but also slices the relevant video segment using FFmpeg.

- **Key Additions**:
  - A `gr.Interface`-based UI for input and video preview.
  - A function `slice_video_clip` that uses FFmpeg to trim the original video using matched timestamps.
  - File lookup logic to find the correct `.mp4` from the dataset.
- **Workflow**:
  1. The user submits a query.
  2. Top semantic match is retrieved.
  3. Corresponding video is located and clipped.
  4. Both transcript and video clip are returned to the user in the UI.

> 🎯 This step bridges text search with visual feedback, enabling grounded retrieval over lecture videos.


In [29]:
import os
import gradio as gr
import subprocess
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient

# Initialize model and Qdrant
model = SentenceTransformer("all-MiniLM-L6-v2")
qdrant = QdrantClient(host="localhost", port=6333)

# Paths
dataset_dir = r"C:\Users\sahil\Downloads\AIVideos\processed_data"
output_dir = r"C:\Users\sahil\Downloads\AIVideos\clips_output"
os.makedirs(output_dir, exist_ok=True)

def find_video_path(root_dir, video_key):
    for root, _, files in os.walk(root_dir):
        for f in files:
            if f == f"{video_key}.mp4":
                return os.path.join(root, f)
    return None

def slice_video_clip(video_path, start, end, output_path):
    cmd = [
        "ffmpeg", "-ss", start, "-to", end,
        "-i", video_path,
        "-c:v", "copy", "-c:a", "copy", "-y",
        output_path
    ]
    subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return output_path

def search_and_clip(query):
    embedding = model.encode(query).tolist()
    results = qdrant.search("video_embeddings", query_vector=embedding, limit=1)
    if not results:
        return "No matches found.", None

    top = results[0].payload
    video_key = top["video_key"]
    start, end = top["start"], top["end"]
    text = top["text"]

    video_path = find_video_path(dataset_dir, video_key)
    if not video_path:
        return f"Video not found: {video_key}", None

    output_path = os.path.join(output_dir, f"{video_key}_clip.mp4")
    slice_video_clip(video_path, start, end, output_path)
    return text, output_path

gr.Interface(
    fn=search_and_clip,
    inputs=gr.Textbox(label="Search Query"),
    outputs=[gr.Textbox(label="Transcript"), gr.Video(label="Video Clip")],
    title="Video Search Assistant",
    description="Ask a question, get the matching video segment."
).launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


This version expands the Gradio interface into a full **video QA assistant**, incorporating **multi-result retrieval, summarization via Ollama (Gemma 3)**, and a more polished UI layout.

- **Enhancements**:
  - Multiple transcript matches are returned as a "Chain of Thoughts".
  - These are passed to Ollama for summarization based on the query.
  - The summary and associated clip are returned together.
  - Refined Gradio layout using `gr.Blocks`, `gr.Row`, and `gr.Column` for better presentation.

- **New Features**:
  - Custom summarization via a local LLM (Gemma 3) through Ollama's REST API.
  - Improved formatting and user interaction: "Clear" button, multi-line display boxes, titles and subtitles.

> 🧠 This turns the pipeline into a usable research or demo tool — ideal for answering deep lecture questions with explainable tracebacks and grounded visual context.


In [2]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance
import os
import subprocess
import requests

# Initialize model and database
model = SentenceTransformer("all-MiniLM-L6-v2")
qdrant = QdrantClient(host="localhost", port=6333)

VIDEO_ROOT = r"C:\Users\sahil\Downloads\AIVideos\processed_data"
CLIP_OUTPUT = r"C:\Users\sahil\Downloads\AIVideos\clips_output"
os.makedirs(CLIP_OUTPUT, exist_ok=True)

# Utility to locate video by key
def find_video_path(video_key):
    for root, _, files in os.walk(VIDEO_ROOT):
        for f in files:
            if f == f"{video_key}.mp4":
                return os.path.join(root, f)
    return None

# Extract clip from full video
def slice_clip(video_path, start, end, output_path):
    cmd = ["ffmpeg", "-ss", start, "-to", end, "-i", video_path, "-c:v", "copy", "-c:a", "copy", "-y", output_path]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return output_path if result.returncode == 0 else None

# Summarize using Ollama with Gemma 3
def summarize_with_ollama(context, question):
    payload = {
        "model": "gemma3",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant summarizing lecture content."},
            {"role": "user", "content": f"Here's what the video said:\n\n{context}\n\nNow, answer this question as clearly as possible:\n{question}"}
        ],
        "stream": False
    }
    try:
        res = requests.post("http://localhost:11434/api/chat", json=payload)
        res.raise_for_status()
        data = res.json()
        if "message" in data and "content" in data["message"]:
            return data["message"]["content"].strip()
        elif "response" in data:
            return data["response"].strip()
        else:
            return f"(Unexpected format: {data})"
    except Exception as e:
        return f"(Failed to summarize using Ollama: {e})"

# Main response function
def respond(query):
    embedding = model.encode(query).tolist()
    results = qdrant.search(collection_name="video_embeddings", query_vector=embedding, limit=3)

    if not results:
        return query, "No matches found.", "N/A", None

    thoughts = "\n\n".join([f"{i+1}. {r.payload['text']}" for i, r in enumerate(results)])
    summary = summarize_with_ollama(thoughts, query)

    top = results[0].payload
    video_key = top["video_key"]
    video_path = find_video_path(video_key)
    clip_path = os.path.join(CLIP_OUTPUT, f"{video_key}_clip.mp4")
    if video_path and not os.path.exists(clip_path):
        clip_path = slice_clip(video_path, top["start"], top["end"], clip_path)

    return query, thoughts, summary, clip_path

# UI Layout
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("🎓 **AI Video Chat Assistant**")
    gr.Markdown("Ask a question. See transcript matches, a smart summary, and watch the related clip.")

    with gr.Row():
        with gr.Column(scale=1):
            input_box = gr.Textbox(label="Ask a question based on the videos", lines=2, placeholder="E.g., What is a rational agent?")
            submit_btn = gr.Button("Submit")
            clear_btn = gr.Button("Clear")

        with gr.Column(scale=1):
            question_output = gr.Textbox(label="Question", lines=2, interactive=False)
            thoughts_output = gr.Textbox(label="Chain of Thoughts (Transcript Matches)", lines=8, interactive=False)
            summary_output = gr.Textbox(label="Final Answer (Summarized by Ollama)", lines=4, interactive=False)

    with gr.Row():
        video_output = gr.Video(label="🎬 Relevant Video Clip", format="mp4")

    # Hooks
    def clear_all():
        return "", "", "", None

    submit_btn.click(respond, inputs=input_box, outputs=[question_output, thoughts_output, summary_output, video_output])
    clear_btn.click(clear_all, inputs=None, outputs=[input_box, question_output, thoughts_output, summary_output, video_output])

demo.launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## ✅ Conclusion

In this project, we successfully designed and implemented a complete **Retrieval-Augmented Generation (RAG)** system for querying educational video content. All components were built from scratch, without relying on high-level frameworks, to deepen our understanding of the architecture and internal workings of RAG systems.

We completed the following key milestones:

- **Data Extraction and Alignment**: Extracted subtitle segments from `.vtt` files and aligned them with the corresponding video frames.
- **Semantic Sentence Merging**: Merged short subtitle segments using `spaCy` into longer, coherent sentences while preserving timestamps.
- **Featurization**: Embedded the merged sentences using `all-MiniLM-L6-v2` and stored them in Qdrant along with metadata.
- **Retrieval Pipeline**: Implemented semantic search using Qdrant and used the timestamps of the top matches to extract precise video clips with `ffmpeg`.
- **Summarization**: Used the local Ollama instance running Gemma 3 to summarize the top transcript matches into concise answers.
- **Deployment**: Built a Gradio web application that:
  - Accepts user queries.
  - Returns relevant transcript matches (Chain of Thoughts).
  - Summarizes responses using a local LLM.
  - Displays the associated video clip inline.

This pipeline demonstrates an efficient, explainable, and multimodal RAG architecture that can be adapted to any domain requiring grounded, video-based question answering.
